In [1]:
%load_ext autoreload
# This line loads the extension 'autoreload' that enables to reload automatically the imported moduls whenever they
# are modified.

%autoreload 2
# This line activates mode 2, meaning:
# automatically reload all the imported moduls before every command

#e.g.
# When you're developing a Python module (my_module.py) and you import it in a notebook:
# from my_module import my_function
# If you then modify my_function in the my_module.py file, you would normally need to restart the kernel or re-import the module for the changes to take effect. 
# However, with %autoreload 2, the module is reloaded automatically, and you can use the updated version right away.

In [2]:
import re

import dill
import pandas as pd

import unimib_snowit_project.utils as u

# Setup

In [3]:
# Base Params

# folder
DATA_IN_DIR = 'data_input'

# files
USERS_IN_FILENAME = 'users.csv'
PROFILES_IN_FILENAME = 'profiles.csv'
CARDS_IN_FILENAME = 'cards.csv'
ORDERS_IN_FILENAME = 'orders.csv'
ORDER_DETAILS_IN_FILENAME = 'order_details.csv'

# OSS: "pickle" stands for .pkl, a typical Python extension to save dataframes and files, similar to .csv, but more efficient.

# folder (to store pickle data after cleaning)
DATA_PKL_DIR = 'data_loaded'

# files (pickle data after cleaning)
USERS_PKL_FILENAME = 'users.pkl'
PROFILES_PKL_FILENAME = 'profiles.pkl'
CARDS_PKL_FILENAME = 'cards.pkl'
ORDERS_PKL_FILENAME = 'orders.pkl'
ORDER_DETAILS_PKL_FILENAME = 'order_details.pkl'

# na values
NA_VALUES = ['', ' ', '""',
             '#N/A', '#N/A N/A', '#NA', 'N/A', '<NA>', 'n/a', # 'NA',
             '-1.#IND', '1.#IND',
             '-1.#QNAN', '-NaN', '-nan', '-NAN', '1.#QNAN', 'NaN', 'nan', 'NAN',
             'NULL', 'Null', 'null',
             'NONE', 'None', 'none',
             '[]'
             ]

In [4]:
# Base paths

root_dir_path = u.get_root_dir()
# in my case: ('C:/Users/davyt/Desktop/Bicocca/Marketing Analytics/unimib_snowit_project')

data_in_dir_path = root_dir_path.joinpath(DATA_IN_DIR) # in my case: ('C:/Users/davyt/Desktop/Bicocca/Marketing Analytics/unimib_snowit_project/data_input')
users_in_path = data_in_dir_path.joinpath(USERS_IN_FILENAME)
profiles_in_path = data_in_dir_path.joinpath(PROFILES_IN_FILENAME)
cards_in_path = data_in_dir_path.joinpath(CARDS_IN_FILENAME)
orders_in_path = data_in_dir_path.joinpath(ORDERS_IN_FILENAME)
order_details_in_path = data_in_dir_path.joinpath(ORDER_DETAILS_IN_FILENAME) 

data_pkl_dir_path = root_dir_path.joinpath(DATA_PKL_DIR) 
users_pkl_path = data_pkl_dir_path.joinpath(USERS_PKL_FILENAME)
profiles_pkl_path = data_pkl_dir_path.joinpath(PROFILES_PKL_FILENAME)
cards_pkl_path = data_pkl_dir_path.joinpath(CARDS_PKL_FILENAME)
orders_pkl_path = data_pkl_dir_path.joinpath(ORDERS_PKL_FILENAME)
order_details_pkl_path = data_pkl_dir_path.joinpath(ORDER_DETAILS_PKL_FILENAME)

# LOAD

## Load Users

In [5]:
safeload_users_df = pd.read_csv(users_in_path,
                                dtype='string', 
                                na_values=[],     
                                keep_default_na=False 
                                )

In [6]:
safeload_users_df.columns

Index(['user.uid', 'createdAt', 'source', 'isAnonymous', 'referralsCount',
       'city', 'language', 'googleId', 'appleId', 'facebookId',
       'referral.medium', 'referral.source', 'referral.type',
       'favouriteZones'],
      dtype='object')

In [7]:
safeload_users_df.shape

(728598, 14)

In [8]:
col_to_check = 'user.uid'
safeload_users_df[col_to_check].unique().shape
# I expect this number to be the same as the dimension of the shape, and that's what happens

(728598,)

In [9]:
col_to_check = 'source'
safeload_users_df[col_to_check].unique()

<StringArray>
['website', 'app', 'terze_parti', 'welfare']
Length: 4, dtype: string

In [10]:
col_to_check = 'isAnonymous'
safeload_users_df[col_to_check].unique()

<StringArray>
['False', 'True', '']
Length: 3, dtype: string

In [11]:
col_to_check = 'referralsCount'
safeload_users_df[col_to_check].unique()

<StringArray>
[  '0.0',      '',   '4.0',   '1.0',   '2.0',   '3.0',   '5.0',   '7.0',
   '6.0',  '40.0',  '30.0',   '8.0',  '10.0',  '87.0',  '15.0',  '27.0',
  '17.0',  '23.0',  '16.0',  '19.0', '100.0',  '38.0',   '9.0']
Length: 23, dtype: string

In [12]:
col_to_check = 'city'
safeload_users_df[col_to_check].unique()

<StringArray>
[             'Aachen',             'Aalborg',            'Aalborg ',
               'Aalen',            'Aalsmeer',               'Aalst',
              'Aalter',               'Aarau',             'Aarburg',
              'Aarhus',
 ...
  'Zvolensk√° Slatina',               'Zwaag',             'Zwickau',
         'Zwijndrecht',           'Zwillikon',         'Zwingenberg',
              'Zwolle',            'Zyrardow',              'Zywiec',
 '\uf8ffüá¨\uf8ffüáß ']
Length: 22006, dtype: string

In [13]:
# I have an encoding problem, for example:
safeload_users_df.groupby('city').count().tail(55).index

Index(['zurigo', 'zutigo', 'ÊÑèÂ§ßÂà©Á±≥ÂÖ∞ÁúÅÁ±≥ÂÖ∞', 'Î∞ÄÎùºÎÖ∏',
       'Ïù¥ÌÉàÎ¶¨ÏïÑ', 'ÿ¨ÿ®ŸäŸÑ', 'ÿßŸÑÿ±ŸäÿßŸÜ', 'ŒëŒ∏ŒÆŒΩŒ±',
       'ŒõŒ¨œÅŒΩŒ±Œ∫Œ±', 'ŒõŒµœÖŒ∫œâœÉŒØŒ±', 'ŒöŒüŒúŒüŒ§ŒóŒùŒó',
       'ŒöœÖœÄŒµœÅŒøœçŒΩœÑŒ±', 'Œ†Œ¨œÑœÅŒ±', 'Œ†Œ¨œÜŒøœÇ', 'ƒ¶amrun',
       'ƒÄda≈æi', 'ƒåavisov', 'ƒåaƒçak', 'ƒåejƒç', 'ƒåel√°kovice',
       'ƒåel√°kovice ', 'ƒåeperka', 'ƒåernihiv', 'ƒåernovice', 'ƒåerno≈°ice',
       'ƒåerno≈°ice ', 'ƒåern√Ω D≈Øl', 'ƒåerven√© Janovice',
       'ƒåerven√© Peƒçky', 'ƒåerven√Ω Kostelec', 'ƒåerven√Ω √öjezd',
       'ƒåerƒçany', 'ƒåesk√© Budƒõjovice', 'ƒåesk√© Budƒõjovice ',
       'ƒåesk√© Velenice', 'ƒåesk√°', 'ƒåesk√° Kamenice', 'ƒåesk√° L√≠pa',
       'ƒåesk√° Republika', 'ƒåesk√° Skalice', 'ƒåesk√° T≈ôebov√°',
       'ƒåesk√Ω Brod', 'ƒåesk√Ω Ji≈ôet√≠n', 'ƒåesk√Ω Krumlov',
       'ƒåesk√Ω Rudolec', 'ƒåesk√Ω Tƒõ≈°√≠n', 'ƒåisovice', 'ƒåty≈ôkoly',
       'ƒå√°slav', 'ƒå√°slavice', 'ƒéurkov', 'ƒ∑ekava', 'ƒ∞STANBUL',
       'ƒ∞stanbul', 'üá¨üáß '],
      dt

In [14]:
# to install the package ftfy trying to solve the encoding problem:
# (MarketingAnalytics) C:\Users\davyt\Desktop\Bicocca\Marketing Analytics> cd unimib_snowit_project
# (MarketingAnalytics) C:\Users\davyt\Desktop\Bicocca\Marketing Analytics\unimib_snowit_project> pdm add ftfy

In [15]:
import pandas as pd
from ftfy import fix_text

safeload_users_df.loc[safeload_users_df['city'].notna(), 'city'] = safeload_users_df.loc[safeload_users_df['city'].notna(), 'city'].apply(fix_text)

In [16]:
safeload_users_df.groupby('city').count().tail(55).index

Index(['zurigo', 'zutigo', 'Ādaži', 'Čavisov', 'Čačak', 'Čejč', 'Čelákovice',
       'Čelákovice ', 'Čeperka', 'Černihiv', 'Černovice', 'Černošice',
       'Černošice ', 'Černý Důl', 'Červené Janovice', 'Červené Pečky',
       'Červený Kostelec', 'Červený Újezd', 'Čerčany', 'Česká',
       'Česká Kamenice', 'Česká Lípa', 'Česká Republika', 'Česká Skalice',
       'Česká Třebová', 'České Budějovice', 'České Budějovice ',
       'České Velenice', 'Český Brod', 'Český Jiřetín', 'Český Krumlov',
       'Český Rudolec', 'Český Těšín', 'Čisovice', 'Čtyřkoly', 'Čáslav',
       'Čáslavice', 'Ďurkov', 'Ħamrun', 'İSTANBUL', 'İstanbul', 'ķekava',
       'Αθήνα', 'ΚΟΜΟΤΗΝΗ', 'Κυπερούντα', 'Λάρνακα', 'Λευκωσία', 'Πάτρα',
       'Πάφος', 'الريان', 'جبيل', '意大利米兰省米兰', '밀라노', '이탈리아', '🇬🇧 '],
      dtype='string', name='city')

In [17]:
safeload_users_df['city']= safeload_users_df['city'].apply(lambda x:
                                                                       u.clean_str(x, 'lower')
                                                                       if pd.notnull(x)
                                                                       else None
                                                                       )

In [18]:
safeload_users_df['city'].unique()
# we reduce the number of unique cities from 22006 to 15862

array(['aachen', 'aalborg', 'aalen', ..., 'zyrardow', 'zywiec', '🇬🇧'],
      shape=(15863,), dtype=object)

In [19]:
col_to_check = 'language'
safeload_users_df[col_to_check].unique()

<StringArray>
['en', 'de', 'it',   '', 'nl', 'es', 'fr', 'cs', 'ru', 'zh', 'sv', 'da', 'fi',
 'pt', 'ro', 'sl', 'nb', 'sk', 'pl']
Length: 19, dtype: string

In [20]:
col_to_check = 'googleId'
safeload_users_df[col_to_check].unique()

<StringArray>
['True', '']
Length: 2, dtype: string

In [21]:
col_to_check = 'appleId'
safeload_users_df[col_to_check].unique()

<StringArray>
['', 'True']
Length: 2, dtype: string

In [22]:
col_to_check = 'facebookId'
safeload_users_df[col_to_check].unique()

<StringArray>
['', 'True']
Length: 2, dtype: string

In [23]:
col_to_check = 'referral.medium'
safeload_users_df[col_to_check].unique()

<StringArray>
[                                 '',                               'cpc',
                           'website',                             'email',
                       'affiliation',             'satispay-landing-page',
                           'cpc,cpc',                     'gite-in-treno',
                           'organic',        'pubbliredazionale-dicembre',
                       'pickuppoint',                          'referral',
                               'app', 'bcg-landing-page,bcg-landing-page',
                             'flyer',                      'landing-page',
                'influencer-stories',                            'banner',
                            'bobbio',                           'sitoweb',
                               'NEN',                      'bcgplatinion',
                        'bormiopass',                              'sito',
                           'bottone',                          'magazine',
           

In [24]:
col_to_check = 'referral.source'
safeload_users_df[col_to_check].unique()

<StringArray>
[                              '',                         'google',
                           'meta',         'piani-di-bobbio-barzio',
                          'brevo',                       'magazine',
                         'widget',                         'social',
                       'satispay',                  'google,google',
                        'trenord',                         'chiesa',
                         'aprica',              'chiesa-valmalenco',
                           'yome',                 'ActiveCampaign',
                     'dovesciare',                    'pickuppoint',
                   'email_snowit',                           'blog',
                        'adwords', 'piani-di-bobbio-barzio,trenord',
            'ads_bikeit_noleggio',                    'mail_snowit',
                     'domobianca',                       'app_home',
                        'skiinfo',                     'FB_funcard',
                    

In [25]:
col_to_check = 'referral.type'
safeload_users_df[col_to_check].unique()

<StringArray>
[     '',   '1.0',   '2.0',   '3.0',   '4.0',   '5.0',   '6.0',   '7.0',
   '8.0',   '9.0',
 ...
 '453.0', '454.0', '455.0', '456.0', '457.0', '458.0', '459.0', '460.0',
 '461.0', '462.0']
Length: 463, dtype: string

In [26]:
col_to_check = 'favouriteZones'
safeload_users_df[col_to_check].unique()

# I obtain that there are 1627 different combinations of unique "at most 5 most preferred locations",
# beware that some of them (as the first one) are empty due to null values not managed yet.


<StringArray>
[                                                                                          '',
                             '[Alagna - Monterosa Ski,Gressoney-La-Trinit√© - Monterosa Ski]',
                                                                                     '[Pila]',
                                                     '[Chiesa Valmalenco Bernina Ski Resort]',
                                                            '[Cervinia,Courmayeur,La Thuile]',
  '[Cervinia,Champoluc - Monterosa Ski,La Thuile,Gressoney-La-Trinit√© - Monterosa Ski,Pila]',
                                                                 '[Barzio - Piani di Bobbio]',
                                                                            '[Alpe di Siusi]',
                                                                                   '[Andalo]',
                                               '[Folgaria,Maniva,Pontedilegno-Tonale,Andalo]',
 ...
 "[Barzio - Piani di Bobbio,Cer

In [27]:
safeload_users_df.groupby('favouriteZones').count().sort_values(by="city", ascending=False).head(10)

,user.uid,createdAt,source,isAnonymous,referralsCount,city,language,googleId,appleId,facebookId,referral.medium,referral.source,referral.type
favouriteZones,,,,,,,,,,,,,
,723967,723967,723967,723967,723967,325386,723967,723967,723967,723967,723967,723967,723967
[Aprica],203,203,203,203,203,192,203,203,203,203,203,203,203
[Pontedilegno-Tonale],189,189,189,189,189,162,189,189,189,189,189,189,189
[Barzio - Piani di Bobbio],165,165,165,165,165,151,165,165,165,165,165,165,165
[Chiesa Valmalenco Bernina Ski Resort],155,155,155,155,155,136,155,155,155,155,155,155,155
[Pila],126,126,126,126,126,117,126,126,126,126,126,126,126
[Cervinia],127,127,127,127,127,111,127,127,127,127,127,127,127
[Madesimo],114,114,114,114,114,106,114,114,114,114,114,114,114
[Folgaria],126,126,126,126,126,105,126,126,126,126,126,126,126


In [28]:
# Take a look at the most appreciated combinations
# Beware, that is not the best way to treat them, I'd like to have a counter for each specific location, but that is 
# something I'll deal with during the Exploratory Data Analysis...

In [29]:
safeload_users_df.groupby('favouriteZones').count().sort_values(by="city", ascending=False).tail(10)

,user.uid,createdAt,source,isAnonymous,referralsCount,city,language,googleId,appleId,facebookId,referral.medium,referral.source,referral.type
favouriteZones,,,,,,,,,,,,,
"[Alagna - Monterosa Ski,Champoluc - Monterosa Ski,Gressoney-La-Trinit√© - Monterosa Ski,La Thuile]",1,1,1,1,1,0,1,1,1,1,1,1,1
"[Alagna - Monterosa Ski,Cervinia,Champoluc - Monterosa Ski,Gressoney-La-Trinit√© - Monterosa Ski,Pila]",1,1,1,1,1,0,1,1,1,1,1,1,1
"[Alagna - Monterosa Ski,Cervinia,Champoluc - Monterosa Ski,Gressoney-La-Trinit√© - Monterosa Ski]",1,1,1,1,1,0,1,1,1,1,1,1,1
"[Alagna - Monterosa Ski,Cervinia,Champoluc - Monterosa Ski,La Thuile,Pila]",1,1,1,1,1,0,1,1,1,1,1,1,1
"[Alagna - Monterosa Ski,Barzio - Piani di Bobbio,La Thuile,Domobianca]",1,1,1,1,1,0,1,1,1,1,1,1,1
"[Alagna - Monterosa Ski,Barzio - Piani di Bobbio,Cervinia,Champoluc - Monterosa Ski,Gressoney-La-Trinit√© - Monterosa Ski]",1,1,1,1,1,0,1,1,1,1,1,1,1
"[Valtorta - Piani di Bobbio,Chiesa Valmalenco Bernina Ski Resort,Foppolo]",1,1,1,1,1,0,1,1,1,1,1,1,1
"[Valtorta - Piani di Bobbio,Foppolo,Monte Pora,Piazzatorre,Selvino]",1,1,1,1,1,0,1,1,1,1,1,1,1
"[Alagna - Monterosa Ski,Alpe di Siusi,Barzio - Piani di Bobbio,Cervinia,La Thuile]",1,1,1,1,1,0,1,1,1,1,1,1,1


In [30]:
# Read and fix the "final" users data
from ftfy import fix_text

users_df = pd.read_csv(users_in_path,
                       keep_default_na=False,
                       na_values=NA_VALUES, # assign the ones previously defined
                       dtype={
                           'user.uid': 'string',
                           'createdAt': 'string',
                           'source': 'string',
                           'isAnonymous': 'boolean',
                           'referralsCount': 'Int64',
                           'city': 'string',
                           'language': 'string',
                           'googleId': 'boolean',
                           'appleId': 'boolean',
                           'facebookId': 'boolean',
                           'referral.medium': 'string',
                           'referral.source': 'string',
                           'referral.type': 'Int64',
                           'favouriteZones': 'string'
                       }
                       )

users_df['createdAt'] = pd.to_datetime(users_df['createdAt'])

users_df['city'] = (users_df['city']
                    .apply(lambda x:
                           u.clean_str(x, 'lower')
                           if pd.notnull(x)
                           else None
                           )
                    )
users_df['city'] = users_df['city'].apply(
    lambda x: fix_text(x) if not (str(x).strip() in NA_VALUES) else x
)


users_df['referral.medium'] = (users_df['referral.medium']
                        .apply(lambda x:
                               u.clean_str(x, 'lower')
                               if pd.notnull(x)
                               else None
                               )
                        )

users_df['referral.source'] = (users_df['referral.source']
                        .apply(lambda x:
                               u.clean_str(x, 'lower')
                               if pd.notnull(x)
                               else None
                               )
                        )

# create an actual list out of the string
users_df['favouriteZones'] = (users_df['favouriteZones']
                              .apply(lambda x:
                                     u.get_list_from_str(x)
                                     if pd.notnull(x)
                                     else []
                                     )
                              )

In [31]:
# CHECK Primary Key VALIDITY

# SELECT count(1) as num_rows
# FROM users_df
# WHERE user.uid IS NULL

display(
    users_df
    .loc[lambda tbl: tbl['user.uid'].isnull()]
    .assign(aux=1.0) 
    .shape[0]
)


# SELECT user.uid, count(1) as num_rows
# FROM users_df
# GROUP BY user.id
# HAVING num_rows > 1

display(
    users_df
    .assign(aux=1.0) # add a new column named 'aux' with value 1, useful to count rows
    .groupby(['user.uid'], dropna=False)
    .agg(num_rows=('aux', pd.Series.count)) # 'agg' creates a new column named 'num_rows' whose values are the result of the application of the function 'pd.Series.count' to the column 'aux'
                                            
    .loc[lambda tbl: tbl['num_rows'] > 1] # lamba used here takes as argument (tbl) the same item I do the .loc on,
                                          # this case the entire dataframe.
                                          # When doing .apply(lambda x) x is a "portion" (like a row) of the item I do,
                                          # for example, the .loc on.
)

0

,num_rows
user.uid,


In [32]:
# example of not having null
tool_data = users_df[1:10]
tool_data.assign(aux=1.0).groupby(['user.uid'], dropna=False).agg(NameNewColumn=('aux', pd.Series.count))

,NameNewColumn
user.uid,
87aezxkbjcfr1zgs6u3dt1cqvl,1
adhwlj2x4rfvugzef8iaijnpjd,1
ez4oclwv92ofoeklnccp4nflgz,1
fol9wcwjvltb0vus6shl0zakfl,1
gg5ntjlexfsaoagecwkvyj7txu,1
mzgjl3z7tfcmmum8k6nanthskk,1
ud3cuumrysslvy5hzgsq6ygovh,1
vgf8cmv8xnp0ifmyugwdzksz3s,1
y35dhzdewid6nzq5ctivwcogxd,1


In [33]:
# A look at the 'final' data
users_df.shape

(728598, 14)

In [34]:
users_df.tail(10)

,user.uid,createdAt,source,isAnonymous,referralsCount,city,language,googleId,appleId,facebookId,referral.medium,referral.source,referral.type,favouriteZones
728588,o2xjmToYS7bQgfVxe7B8zoxGgz3,2024-12-29,website,False,<NA>,zwolle,en,True,<NA>,<NA>,None,None,<NA>,[]
728589,saP3XuuUn4gR22EyBH1dgs5uTHy,2023-12-27,website,False,0,zwolle,<NA>,<NA>,<NA>,<NA>,None,None,<NA>,[]
728590,txpG2bQl8Ve45ANraCVV40JSR71,2022-01-10,website,False,<NA>,zwolle,<NA>,<NA>,<NA>,<NA>,None,None,<NA>,[]
728591,ulOwZeYSFOXLeIS3mXq5NRz1WGK,2025-01-20,website,False,<NA>,zwolle,<NA>,<NA>,<NA>,<NA>,None,None,<NA>,[]
728592,C1IV755uejbd5mVaJh3tsX0GQ1o,2023-02-21,website,False,0,zyrardow,it,True,<NA>,<NA>,None,None,<NA>,[]
728593,14G4T7sZLhSnMkUXmAdOVCuevaL,2023-11-24,website,False,0,zywiec,it,True,<NA>,<NA>,None,None,<NA>,[]
728594,qBYuW5TV7dN1zm7bzGtOzIsGoXv,2025-01-09,website,False,<NA>,🇬🇧,en,<NA>,<NA>,<NA>,None,None,<NA>,[]
728595,QYVsQw7QvTQRFswaLA2nO8v0FhH,2022-01-03,website,False,<NA>,None,it,<NA>,<NA>,<NA>,None,None,<NA>,[]
728596,hKVbfqQpPYXQ28Syz8gP7xmpofL,2024-01-08,website,False,0,None,it,<NA>,<NA>,<NA>,None,None,<NA>,[]
728597,oWRue3fvr5P8S1FCziP1im53XOm,2025-02-14,app,False,<NA>,None,<NA>,<NA>,<NA>,<NA>,None,None,<NA>,[]


## Load Profiles

In [35]:
safeload_profiles_df = pd.read_csv(profiles_in_path,
                                   dtype='string',
                                   na_values=[],
                                   keep_default_na=False
                                   )

In [36]:
safeload_profiles_df.columns

Index(['user.uid', 'profile.uid', 'birthday', 'sex', 'city', 'height',
       'weight', 'skibootsSize', 'level', 'types'],
      dtype='object')

In [37]:
safeload_profiles_df.shape

(97158, 10)

In [38]:
col_to_check = 'profile.uid'
safeload_profiles_df[col_to_check].unique()
# that's the primary key and, as expected, has the same dimension of the dataset

<StringArray>
['9oqhfw0zsc5prlkwh9gk', 'miejgay5t0m7v7gsjffo', 'vi88jcuymkmxkandvy3l',
 'p54s6m0hejd4nucibz4e', 'zkzpf8fnha6s4kmc2rwn', 'bqnrhm4itmwhofvxxxqh',
 'gci1tdbxhtyvcdacfewi', 'mmnf0ffbevaspno2u6zf', 'sw6ezr1tlxcpndhgka1s',
 'nuuzmulrxdbrhww3o70q',
 ...
 '1r2dtjklawjzgnbfcvdr', 'mmrzndbjrwu9rycgzhtp', 'hvj0s7frv7wuwgohq1hh',
 'le259lscdxhmvwjgfpym', 'balfq1j0nsnlztp2ntjh', 'rrjectzobab0e0yjg7ln',
 'c1xmzeeq0shd5wq5j3mk', 'miu7ipqiuvwnmdjrcfbj', 'i5jcva8arghxpflohvra',
 'qve3xjgnvnhqtevfa0sf']
Length: 97158, dtype: string

In [39]:
col_to_check = 'user.uid'
safeload_profiles_df[col_to_check].unique()
# oss 'profiles' > 'user', meaning that more profiles (mother, father, daughter) are related to the same user account

<StringArray>
['tvvgjd4sdgfucn33brkigrnjjv', 'iwu2obhsgzamrxcp4wnoxlf5b0',
 'bmgilq4eatrpt2hhwamnbcqnir', 'ysbrr5ve9azqvr1jw72expbnxr',
 'txy3ls535wzclygjnlt2gjalm1', 'f73bpmavwla9pho2qpgi5n7cdx',
 '2ch0jf5weordcmluqz51cuz1vb', 'fi9xtuvfhfqqkm9kcvonh7b13m',
 'nu7wfxo3u3cxbb5zfhelggfg1y', 'iuviiqflcjydzbvj7xbveexauh',
 ...
 '1h5aimn0ioc8z9wdybnvsneub3', 'ajvmewdjmntwl3qe9freo9020j',
 'hx4c5qjla6fwtkdzflhyy3leaa', 'kayyuolrnvrznzlcausph85t9a',
 'z2hubzult5xm8ixddc6xv2otyz', 'j2jgcghmxhbwuocf0koypm27be',
 'nbf8wvwtnrv97vdu1pk00drcxs', 'ucyqnolgulsap6k3835jfeapdk',
 'jzlczkcpaphgkgfljwu1tn3usl', 'mszngxuiuieb06relyhvduaxmq']
Length: 64238, dtype: string

In [40]:
col_to_check = 'birthday'
safeload_profiles_df[col_to_check].unique()

<StringArray>
['2000-05-23', '1994-08-07', '1992-05-06', '1998-02-16', '1994-05-12',
 '1979-12-09', '1982-02-02', '2009-09-18', '2000-03-04', '2004-02-22',
 ...
 '2024-01-17', '1956-09-26', '2016-06-28', '2018-12-08', '1964-09-09',
 '2021-02-25', '1962-02-13', '1959-06-14', '1972-05-09', '2019-03-13']
Length: 19464, dtype: string

In [41]:
col_to_check = 'sex'
safeload_profiles_df[col_to_check].unique()
# will be needed a function to standardize them

<StringArray>
['M', 'F', '', 'Uomo', 'Maschio', '""', 'm', 'maschio']
Length: 8, dtype: string

In [42]:
col_to_check = 'city'
safeload_profiles_df[col_to_check].unique()

<StringArray>
[         'Zwolle',          'Zurigo',         'Z√ºrich',          'Zurich',
       'ZUR MOSHE',        'Zugliano',            'zone',    'Zola Predosa',
    'Zola predosa',   'Zogno Bergamo',
 ...
          'Acerra',       'Abu Dhabi',         'Absecon',       'Aboncourt',
        'Aberdeen',   'Abbiategrasso',   'ABBIATEGRASSO',   'abbiategrasso',
 'Abbadia Lariana',          'Aarhus']
Length: 3227, dtype: string

In [43]:
safeload_profiles_df['city'] = (safeload_profiles_df['city']
                                .apply(lambda x:
                                       u.clean_str(x, 'lower')
                                       if pd.notnull(x)
                                       else None
                                       )
                                )
safeload_profiles_df['city'] = safeload_profiles_df['city'].apply(
    lambda x: fix_text(x) if not (str(x).strip() in NA_VALUES) else x
)

In [44]:
safeload_profiles_df[col_to_check].unique()

array(['zwolle', 'zurigo', 'zürich', ..., 'abbiategrasso',
       'abbadia lariana', 'aarhus'], shape=(2464,), dtype=object)

In [45]:
col_to_check = 'height'
safeload_profiles_df[col_to_check].unique()

<StringArray>
[   '180.0',    '185.0',         '',    '161.0',    '159.0',    '165.0',
    '158.0',    '200.0',    '160.0',    '178.0',
 ...
     '42.0',   '1890.0', '501660.0',    '211.0',     '44.0',     '51.0',
      '6.0',     '61.0',   '1779.0',     '17.0']
Length: 191, dtype: string

In [46]:
import pandas as pd

safeload_profiles_df['height'] = pd.to_numeric(safeload_profiles_df['height'], errors='coerce')
summary = safeload_profiles_df['height'].describe()

print(summary)


count        75771.0
mean      174.904566
std      1822.869884
min              1.0
25%            163.0
50%            171.0
75%            179.0
max         501660.0
Name: height, dtype: Float64


In [47]:
count = (safeload_profiles_df['height'] > 250).sum()
print(f"Values with height > 250cm: {count}")

Values with height > 250cm: 9


In [48]:
import numpy as np
safeload_profiles_df.loc[safeload_profiles_df['height'] > 250, 'height'] = np.nan


In [49]:
count = (safeload_profiles_df['height'] < 50).sum()
print(f"Values with height < 50cm: {count}")

Values with height < 50cm: 40


In [50]:
safeload_profiles_df.loc[safeload_profiles_df['height'] < 50, 'height'] = np.nan

In [51]:
col_to_check = 'weight'
safeload_profiles_df[col_to_check].unique()

<StringArray>
[  '70',   '95',     '',   '56',   '80',   '75',   '55',  '110',   '68',
   '72',
 ...
  '179',  '146', '8043',  '260',   '10',  '152',  '186', '3090',  '134',
 '5963']
Length: 201, dtype: string

In [52]:
safeload_profiles_df['weight'] = pd.to_numeric(safeload_profiles_df['weight'], errors='coerce')
summary = safeload_profiles_df['weight'].describe()

print(summary)

count      75548.0
mean     65.756871
std      42.012059
min            0.0
25%           55.0
50%           65.0
75%           77.0
max         8043.0
Name: weight, dtype: Float64


In [53]:
count = (safeload_profiles_df['weight'] > 160).sum()
print(f"Values with weight > 160kg: {count}")

#let's assume these people entered their weight in libbre

Values with weight > 160kg: 147


In [54]:
def correct_weight_column(df, col='weight'):
    df[col] = pd.to_numeric(df[col], errors='coerce') 
    df[col] = df[col].apply(lambda w: w / 2.205 if w > 160 else w)
    
    return df


In [55]:
safeload_profiles_df = correct_weight_column(safeload_profiles_df)

In [56]:
count = (safeload_profiles_df['weight'] < 13).sum()
print(f"Values with weight < 13kg: {count}")

Values with weight < 13kg: 16


In [57]:
col_to_check = 'skibootsSize'
safeload_profiles_df[col_to_check].unique()

<StringArray>
[  '43',     '',   '38',   '45',   '40',   '48',   '44',   '42',   '36',
   '41',
 ...
 '10.5',  '248',  '165',   '80',  '318',  '355',   '16',  '160', '4037',
  '338']
Length: 130, dtype: string

In [58]:
safeload_profiles_df['skibootsSize'] = pd.to_numeric(safeload_profiles_df['skibootsSize'], errors='coerce')
summary = safeload_profiles_df['skibootsSize'].describe()

print(summary)

count      75526.0
mean     41.036941
std      48.518657
min            0.0
25%           38.0
50%           41.0
75%           43.0
max         4546.0
Name: skibootsSize, dtype: Float64


In [59]:
col_to_check = 'level'
safeload_profiles_df[col_to_check].unique()

<StringArray>
['beginner', 'intermediate', 'expert', '']
Length: 4, dtype: string

In [60]:
col_to_check = 'types'
safeload_profiles_df[col_to_check].unique()

# out of the 97158 profiles (before cleaning), I have 129 unique combinations of available services

<StringArray>
[                             '["insurance","rent","rent/ski","rent/bike","lesson","lesson/ski","lesson/bike","hotel","travel","train","transport","itinerary","skipass"]',
                                         '["insurance","rent/bike","rent","lesson","lesson/ski","lesson/bike","hotel","travel","train","transport","itinerary","skipass"]',
           '["insurance","rent","rent/ski","rent/bike","lesson","lesson/ski","lesson/bike","hotel","travel","train","transport","itinerary","skipass","skipass/promo-cf"]',
                             '["insurance","rent","rent/ski","rent/bike","lesson","lesson/ski","lesson/bike","hotel","travel","train","transport","itinerary","seasonal"]',
           '["insurance","rent/bike","rent","lesson","lesson/ski","lesson/bike","hotel","travel","train","transport","itinerary","seasonal/maestri","seasonal","skipass"]',
                                                                           '["insurance","rent/bike","rent","train","transport

In [61]:
# Read and fix
import numpy as np

profiles_df = pd.read_csv(profiles_in_path,
                       keep_default_na=False,
                       na_values=NA_VALUES, # assign the ones previously defined
                       dtype={
                           'user.uid': 'string',
                           'profile.uid': 'string',
                           'birthday': 'string',
                           'sex': 'string',
                           'city': 'string',
                           'height': 'Float64',
                           'weight': 'Float64',
                           'skibootsSize': 'Float64',
                           'level': 'string',
                           'types': 'string'
                       }
                       )

profiles_df['birthday'] = pd.to_datetime(profiles_df['birthday'])

def clean_profile_sex(sex: str) -> str | None:
    clean = u.clean_str(sex, 'upper')
    if clean in ['M', 'F']:
        return clean
    elif clean == ['UOMO', 'MASCHIO']:
        return 'M'
    elif clean == ['DONNA', 'FEMMINA']:
        return 'F'
    else:
        None
profiles_df['sex'] = (profiles_df['sex']
                      .apply(lambda x:
                             clean_profile_sex(x)
                             if pd.notnull(x)
                             else None
                             )
                      )

profiles_df['city'] = (profiles_df['city']
                    .apply(lambda x:
                           u.clean_str(x, 'lower')
                           if pd.notnull(x)
                           else None
                           )
                    )
profiles_df['city'] = profiles_df['city'].apply(
    lambda x: fix_text(x) if not (str(x).strip() in NA_VALUES) else x
)

profiles_df.loc[ (safeload_profiles_df['height'] < 50) | (safeload_profiles_df['height'] > 250),
                'height'] = np.nan

def correct_weight_column(df, col='weight'):
    df[col] = pd.to_numeric(df[col], errors='coerce') 
    df[col] = df[col].apply(lambda w: w / 2.205 if w > 160 else w)
    df.loc[df[col] < 13, col] = np.nan
    return df
profiles_df = correct_weight_column(profiles_df)

profiles_df['types'] = (profiles_df['types']
                              .apply(lambda x:
                                     u.get_list_from_str(x)
                                     if pd.notnull(x)
                                     else []
                                     )
                              )

In [62]:
# CHECK Primary Key VALIDITY

# SELECT count(1) as num_rows
# FROM profiles_df
# WHERE profile.uid IS NULL

display(
    profiles_df
    .loc[lambda tbl: tbl['profile.uid'].isnull()]
    .assign(aux=1.0)
    .shape[0]
)

# SELECT profile.uid, count(1) as num_rows
# FROM profiles_df
# GROUP BY profile.id
# HAVING num_rows > 1

display(
    profiles_df
    .assign(aux=1.0)
    .groupby(['profile.uid'], dropna=False)
    .agg(num_rows=('aux', pd.Series.count))
    .loc[lambda tbl: tbl['num_rows'] > 1]
)

0

,num_rows
profile.uid,


In [63]:
# CHECK Foreign Key VALIDITY

# SELECT
#   A.user.uid,
#   count(1) as num_rows
# FROM 
#   (SELECT DISTINCT user.uid
#   FROM profiles_df
#   WHERE user.uid IS NOT NULL) AS A
#   LEFT JOIN
#   (SELECT user.uid, 1.0 AS in_users
#   FROM users_df) AS B
#   ON A.user.uid = B.user.uid
# GROUP BY in_users
# HAVING num_rows > 1

(profiles_df
 [['user.uid']]
 .loc[lambda tbl: tbl['user.uid'].notnull()]
 .drop_duplicates()
 .merge(users_df[['user.uid']].assign(in_users=1.0),
        how='left',
        on='user.uid'
        )
 .assign(aux=1.0)
 .groupby(['in_users'], dropna=False)
 .agg(num_rows=('aux', pd.Series.count))
 .loc[lambda tbl: tbl['num_rows'] > 1]
)

,num_rows
in_users,
1.0,46026
NaN,18212


In [64]:
# issue: I have 18212 profiles with a userID that is not matched with an official one ! 

In [65]:
# let's visualize these 18212 profiles

# SELECT
#   A.user.uid,
#   count(1) as num_rows
# FROM 
#   (SELECT DISTINCT user.uid
#   FROM profiles_df
#   WHERE user.uid IS NOT NULL) AS A
#   LEFT JOIN
#   (SELECT user.uid, 1.0 AS in_users
#   FROM users_df) AS B
#   ON A.user.uid = B.user.uid
# WHERE
#   in_users IS NULL

(profiles_df
 [['user.uid']]
 .loc[lambda tbl: tbl['user.uid'].notnull()]
 .drop_duplicates()
 .merge(users_df[['user.uid']].assign(in_users=1.0),
        how='left',
        on='user.uid'
        )
 .loc[lambda tbl: tbl['in_users'].isnull()]
)

,user.uid,in_users
0,tvvgjd4sdgfucn33brkigrnjjv,NaN
2,bmgilq4eatrpt2hhwamnbcqnir,NaN
3,ysbrr5ve9azqvr1jw72expbnxr,NaN
4,txy3ls535wzclygjnlt2gjalm1,NaN
5,f73bpmavwla9pho2qpgi5n7cdx,NaN
...,...,...
64149,mtlnpaw38weptp9pmbbyifknko,NaN
64164,jpkst6ew69gkluh7in43m4ohvs,NaN
64166,smqgdbfhyofngnjqth4bglwax7,NaN
64217,pzffry3ps6aagidfvisp89giam,NaN


In [66]:
# from the table printed before I select just the column ['user.uid'],
# those IDs without a correspondence in the users_df
profile_fail_useruids = (profiles_df
    [['user.uid']]
    .loc[lambda tbl: tbl['user.uid'].notnull()]
    .drop_duplicates()
    .merge(users_df[['user.uid']].assign(in_users=1.0),
            how='left',
            on='user.uid'
            )
    .loc[lambda tbl: tbl['in_users'].isnull()]
    ['user.uid']
)
display(profile_fail_useruids)

# extract now the profiles with "inconsistent" 'user.uid'
profile_fail_useruid_df = profiles_df.loc[lambda tbl: tbl['user.uid'].isin(profile_fail_useruids)]
display(profile_fail_useruid_df)

0        tvvgjd4sdgfucn33brkigrnjjv
2        bmgilq4eatrpt2hhwamnbcqnir
3        ysbrr5ve9azqvr1jw72expbnxr
4        txy3ls535wzclygjnlt2gjalm1
5        f73bpmavwla9pho2qpgi5n7cdx
                    ...            
64149    mtlnpaw38weptp9pmbbyifknko
64164    jpkst6ew69gkluh7in43m4ohvs
64166    smqgdbfhyofngnjqth4bglwax7
64217    pzffry3ps6aagidfvisp89giam
64219    34w1omc2f7fskwlhteltqcomrc
Name: user.uid, Length: 18212, dtype: string

,user.uid,profile.uid,birthday,sex,city,height,weight,skibootsSize,level,types
0,tvvgjd4sdgfucn33brkigrnjjv,9oqhfw0zsc5prlkwh9gk,2000-05-23,M,zwolle,180.0,70.0,43.0,beginner,"[""insurance"", ""rent"", ""rent/ski"", ""rent/bike"",..."
2,bmgilq4eatrpt2hhwamnbcqnir,vi88jcuymkmxkandvy3l,1992-05-06,F,zürich,<NA>,NaN,<NA>,intermediate,"[""insurance"", ""rent/bike"", ""rent"", ""lesson"", ""..."
3,ysbrr5ve9azqvr1jw72expbnxr,p54s6m0hejd4nucibz4e,1998-02-16,F,zurich,161.0,56.0,38.0,beginner,"[""insurance"", ""rent"", ""rent/ski"", ""rent/bike"",..."
4,ysbrr5ve9azqvr1jw72expbnxr,zkzpf8fnha6s4kmc2rwn,1994-05-12,M,zurich,180.0,80.0,45.0,beginner,"[""insurance"", ""rent"", ""rent/ski"", ""rent/bike"",..."
5,txy3ls535wzclygjnlt2gjalm1,bqnrhm4itmwhofvxxxqh,1979-12-09,M,zur moshe,180.0,75.0,43.0,beginner,"[""insurance"", ""rent"", ""rent/ski"", ""rent/bike"",..."
...,...,...,...,...,...,...,...,...,...,...
97043,smqgdbfhyofngnjqth4bglwax7,ib1un3kn23p8dn5jso3e,2018-09-04,M,albosaggia,118.0,20.0,29.0,beginner,"[""insurance"", ""rent"", ""rent/ski"", ""rent/bike"",..."
97044,smqgdbfhyofngnjqth4bglwax7,ah904epxoefqvk2slt6k,2018-09-04,M,albosaggia,118.0,20.0,29.0,beginner,"[""insurance"", ""rent"", ""rent/ski"", ""rent/bike"",..."
97097,dyz1ivjmcym14jagwuyjnhtrz7,mdqipvzojgtgsajxjtlx,1999-10-05,None,aidone,155.0,55.0,36.0,<NA>,"[""insurance"", ""rent/bike"", ""rent"", ""travel"", ""..."
97126,pzffry3ps6aagidfvisp89giam,mbpgeblzb2t0bylyrhww,2003-10-17,M,acquedolci,167.0,60.0,39.0,intermediate,"[""insurance"", ""rent"", ""rent/ski"", ""rent/bike"",..."


In [67]:
# FIX Foreing Key ISSUE: remove them ! 
profiles_df.drop(list(profile_fail_useruid_df.index), inplace=True)

In [68]:
display(
    profiles_df
    [['user.uid']]
    .loc[lambda tbl: tbl['user.uid'].notnull()]
    .drop_duplicates()
    .merge(users_df[['user.uid']].assign(in_users=1.0),
            how='left',
            on='user.uid'
            )
    .assign(aux=1.0)
    .groupby(['in_users'], dropna=False)
    .agg(num_rows=('aux', pd.Series.count))
    .loc[lambda tbl: tbl['num_rows'] > 1]
)
# now, as expected, I don't have null values after the left join

,num_rows
in_users,
1.0,46026


## Load Cards

In [69]:
safeload_cards_df = pd.read_csv(cards_in_path,
                                dtype='string',
                                na_values=[],
                                keep_default_na=False
                                )

In [70]:
safeload_cards_df.columns

Index(['card.uid', 'assignedAt', 'birthday', 'status', 'user.uid'], dtype='object')

In [71]:
safeload_cards_df.shape

(805837, 5)

In [72]:
col_to_check = 'card.uid'
safeload_cards_df[col_to_check].unique()
# since this length is lower than the dimension,
# it means that either some ids are null or some ids are not unique

<StringArray>
['00000001', '00000011', '00000022', '00000033', '00000044', '00000055',
 '00000066', '00000077', '00000088', '00000099',
 ...
 '99131935', '99131946', '99131957', '99131968', '99131979', '99131980',
 '99131991', '99999995', '99999997', '99999999']
Length: 710917, dtype: string

In [73]:
col_to_check = 'card.uid'
num_nulls = safeload_cards_df[col_to_check].isnull().sum()
print(f"Null values in '{col_to_check}': {num_nulls}")
# there are some duplicates

Null values in 'card.uid': 0


In [74]:
dupe_key = safeload_cards_df['card.uid'].duplicated(keep=False)
dupe_full = safeload_cards_df.duplicated(keep=False)

conflicting_rows = safeload_cards_df[dupe_key & ~dupe_full]
print(conflicting_rows)

# if card.uid has a duplicate, then the full row is a duplicate

Empty DataFrame
Columns: [card.uid, assignedAt, birthday, status, user.uid]
Index: []


In [75]:
col_to_check = 'user.uid'
safeload_cards_df[col_to_check].unique()
# as expected, the number of unique user ids is inferior to the number of cards,
# since the same user can activate cards for others

<StringArray>
['6m8fx3mvanac7bcltkx79lbxjs', 'ogv7xt7089clwmbgmtam35zvjq',
 'naxfhkqjhcqrx3nv6ge9kxenth',                           '',
 '70uymyvsxtusxwrrgva0bfen8q', '9w4drnej3ixwqtwkixk94r7rdn',
 'ujp5e6dtw9cvituayhlvevvdk6', 'mlwkfozqeefazgmboia8hxxkj1',
 'uhxdpccr8npu3lljw9osf0vlwy', '78nvha6ci7nldfizzhmdxaodqr',
 ...
 'mmp6yjfsqbudllov893ysyt3ke', 'ridpfdmnd9oqmqmx2eet14mwzh',
 'nntnw7pjnuqrtb07kkdi09ukvl', 't0yezkfzeuhfhqfmdvdshrvwva',
 'qeoinqsbzbdkho47uudymbmi8q', 'dov2d9ymmdwz1umyvj4c07amox',
 'm3a4kh3q78tvvqidugyqxw0rna', 'a3k15nzfzmrmklnnpkuonacrwf',
 'abpz5y53frcqdfc9nl5ojj1uzi', 'cfui0h9ooqogedmrpsdedq4b7k']
Length: 161029, dtype: string

In [76]:
col_to_check = 'assignedAt'
safeload_cards_df[col_to_check].unique()

<StringArray>
['2025-02-01', '2023-03-13', '2022-01-14',           '', '2018-01-03',
 '2017-12-20', '2018-01-07', '2019-02-08', '2019-12-14', '2019-11-25',
 ...
 '2024-08-29', '2024-07-27', '2024-08-21', '2024-05-13', '2024-08-02',
 '2024-06-07', '2023-04-20', '2024-07-20', '2021-09-17', '2022-06-23']
Length: 1775, dtype: string

In [77]:
col_to_check = 'birthday'
safeload_cards_df[col_to_check].unique()

<StringArray>
['2018-07-02', '1985-02-21', '1985-09-28',           '', '2008-12-31',
 '2004-10-16', '1992-11-12', '1999-05-19', '1975-01-09', '1995-05-08',
 ...
 '1951-09-13', '1950-01-08', '1951-01-19', '1955-11-20', '2019-09-19',
 '1956-04-16', '1963-04-14', '1950-12-19', '1955-02-02', '2018-02-11']
Length: 25266, dtype: string

In [78]:
col_to_check = 'status'
safeload_cards_df[col_to_check].unique()

<StringArray>
[     'pending', 'not-assigned',        'valid',     'rejected',
      'warning',   'membership',        'error']
Length: 7, dtype: string

In [79]:
# Read and fix
import numpy as np

cards_df = pd.read_csv(cards_in_path,
                       keep_default_na=False,
                       na_values=NA_VALUES, # assign the ones previously defined
                       dtype={
                           'user.uid': 'string',
                           'card.uid': 'string',
                           'assignedAt': 'string',
                           'birthday': 'string',
                           'status': 'string',
                       }
                       )

cards_df = cards_df.drop_duplicates()

cards_df['assignedAt'] = pd.to_datetime(cards_df['assignedAt'])

cards_df['birthday'] = pd.to_datetime(cards_df['birthday'], format='mixed')

In [80]:
# CHECK Primary Key VALIDITY

# SELECT count(1) as num_rows
# FROM cards_df
# WHERE cards.uid IS NULL

display(
    cards_df
    .loc[lambda tbl: tbl['card.uid'].isnull()]
    .assign(aux=1.0)
    .shape[0]
)
# SELECT card.uid, count(1) as num_rows
# FROM cards_df
# GROUP BY cards.uid
# HAVING num_rows > 1

display(
    cards_df
    .assign(aux=1.0)
    .groupby(['card.uid'], dropna=False)
    .agg(num_rows=('aux', pd.Series.count))
    .loc[lambda tbl: tbl['num_rows'] > 1]
)

0

,num_rows
card.uid,


In [81]:
# CHECK Foreign Key VALIDITY

# SELECT
#   A.user.uid,
#   count(1) as num_rows
# FROM 
#   (SELECT DISTINCT user.uid
#   FROM cards_df
#   WHERE user.uid IS NOT NULL) AS A
#   LEFT JOIN
#   (SELECT user.uid, 1.0 AS in_users
#   FROM users_df) AS B
#   ON A.user.uid = B.user.uid
# GROUP BY in_users
# HAVING num_rows > 1

(cards_df
 [['user.uid']]
 .loc[lambda tbl: tbl['user.uid'].notnull()]
 .drop_duplicates()
 .merge(users_df[['user.uid']].assign(in_users=1.0),
        how='left',
        on='user.uid'
        )
 .assign(aux=1.0)
 .groupby(['in_users'], dropna=False)
 .agg(num_rows=('aux', pd.Series.count))
 .loc[lambda tbl: tbl['num_rows'] > 1]
)

,num_rows
in_users,
1.0,111687
NaN,49341


In [82]:
# issue, I have 49341 cards with an empty left join with the user.uid table
# let's visualize these 49341 profiles

In [83]:
# SELECT
#   A.user.uid,
#   count(1) as num_rows
# FROM 
#   (SELECT DISTINCT user.uid
#   FROM cards_df
#   WHERE user.uid IS NOT NULL) AS A
#   LEFT JOIN
#   (SELECT user.uid, 1.0 AS in_users
#   FROM users_df) AS B
#   ON A.user.uid = B.user.uid
# WHERE
#   in_users IS NULL

(cards_df
 [['user.uid']]
 .loc[lambda tbl: tbl['user.uid'].notnull()]
 .drop_duplicates()
 .merge(users_df[['user.uid']].assign(in_users=1.0),
        how='left',
        on='user.uid'
        )
 .loc[lambda tbl: tbl['in_users'].isnull()]
)

,user.uid,in_users
0,6m8fx3mvanac7bcltkx79lbxjs,NaN
5,ujp5e6dtw9cvituayhlvevvdk6,NaN
6,mlwkfozqeefazgmboia8hxxkj1,NaN
13,qdre60bu5yqy1socrinaqpwjpw,NaN
20,ip7sunw40nzhy7hhqvoe4pbtvy,NaN
...,...,...
161008,pwdzotpiuumlsgcxnbodsqqmon,NaN
161010,k9meaohmztomnopznzzbcdorqt,NaN
161013,qod9geqy2yyqosu2xrjyatbsvq,NaN
161014,nvezbqfonuzshthd38g3ajjopo,NaN


In [84]:
# from the table printed before I select just the column ['user.uid'],
# those IDs without a correspondence in the users_df
card_fail_useruids = (cards_df
    [['user.uid']]
    .loc[lambda tbl: tbl['user.uid'].notnull()]
    .drop_duplicates()
    .merge(users_df[['user.uid']].assign(in_users=1.0),
            how='left',
            on='user.uid'
            )
    .loc[lambda tbl: tbl['in_users'].isnull()]
    ['user.uid']
)
display(card_fail_useruids)

0         6m8fx3mvanac7bcltkx79lbxjs
5         ujp5e6dtw9cvituayhlvevvdk6
6         mlwkfozqeefazgmboia8hxxkj1
13        qdre60bu5yqy1socrinaqpwjpw
20        ip7sunw40nzhy7hhqvoe4pbtvy
                     ...            
161008    pwdzotpiuumlsgcxnbodsqqmon
161010    k9meaohmztomnopznzzbcdorqt
161013    qod9geqy2yyqosu2xrjyatbsvq
161014    nvezbqfonuzshthd38g3ajjopo
161026    abpz5y53frcqdfc9nl5ojj1uzi
Name: user.uid, Length: 49341, dtype: string

In [85]:
# extract now the profiles with "inconsistent" 'user.uid'
card_fail_useruid_df = cards_df.loc[lambda tbl: tbl['user.uid'].isin(card_fail_useruids)]
display(card_fail_useruid_df)

# FIX Foreing Key ISSUE: remove them ! 
cards_df.drop(list(card_fail_useruid_df.index), inplace=True)

display(
    cards_df
    [['user.uid']]
    .loc[lambda tbl: tbl['user.uid'].notnull()]
    .drop_duplicates()
    .merge(users_df[['user.uid']].assign(in_users=1.0),
            how='left',
            on='user.uid'
            )
    .assign(aux=1.0)
    .groupby(['in_users'], dropna=False)
    .agg(num_rows=('aux', pd.Series.count))
    .loc[lambda tbl: tbl['num_rows'] > 1]
)
# now, as expected, I don't have null values after the left join

,card.uid,assignedAt,birthday,status,user.uid
0,00000001,2025-02-01,2018-07-02,pending,6m8fx3mvanac7bcltkx79lbxjs
7,00000077,2017-12-20,NaT,pending,ujp5e6dtw9cvituayhlvevvdk6
8,00000088,2018-01-07,NaT,pending,mlwkfozqeefazgmboia8hxxkj1
12,00000125,2020-01-02,NaT,pending,ujp5e6dtw9cvituayhlvevvdk6
13,00000136,2020-01-19,NaT,pending,ujp5e6dtw9cvituayhlvevvdk6
...,...,...,...,...,...
805346,99127121,2022-03-11,1990-11-11,pending,oa88nzwvnfqifqhhuegnizcuzk
805347,99127132,2022-12-29,1996-02-21,pending,nvezbqfonuzshthd38g3ajjopo
805369,99127350,2024-12-27,2016-01-01,valid,oa88nzwvnfqifqhhuegnizcuzk
805736,99131027,2024-01-19,1965-04-23,pending,abpz5y53frcqdfc9nl5ojj1uzi


,num_rows
in_users,
1.0,111687


## Load Orders

In [86]:
safeload_orders_df = pd.read_csv(orders_in_path,
                                 dtype='string',
                                 na_values=[],
                                 keep_default_na=False
                                )

In [87]:
safeload_orders_df.columns

Index(['order.uid', 'user.uid', 'createdAt', 'createdAtTime', 'paymentGateway',
       'paymentBrand', 'pickup', 'pickupComplete', 'source', 'tenant',
       'paymentAttempts', 'timeZone', 'clientInfo'],
      dtype='object')

In [88]:
safeload_orders_df.shape

(549900, 13)

In [89]:
col_to_check = 'order.uid'
safeload_orders_df[col_to_check].unique()

<StringArray>
['4y9zqqvldfqr9n2xnu', 'termljwsr2gecnzwks', 'ko8axscf5grobnz9in',
 'tu8b9haokkimu72aza', 'a8mvwuv6ileh9aqblu', 'kmwabdpnhtm46p8me8',
 'i9eovtgp3uxbmesebv', 'nsc55zfspubshggcm5', 'ltr5f48gwicqwcnzkf',
 'eyx8wesrjaeltlgqna',
 ...
 '5f8he6aq0sbnmrrwzq', 'g0tkwbe686jnynbxs9', 'aoud70hexrj3uryxmd',
 'h0vb0rmb2mk4c7s3tv', 'v1obzy9nwp99nanhue', '6icnjfrltzhdi2eato',
 '4n6z8api7q0cz8jxox', 'pdubfidjsudk9mhwdc', 'f8zj28hdatvraibekk',
 'wy69h8jsgozdrxlu9i']
Length: 549900, dtype: string

In [90]:
col_to_check = 'user.uid'
safeload_orders_df[col_to_check].unique()

<StringArray>
['nu7fhz41rzwbkabapcufr6g18r', 'ghol7lwjlhewwvasejlxa9fv00',
 'nhuh3scqoktj6n5mnqkvgyh43h', 'oyekz0wyxcplvrpencpuue6sdt',
 '1p0ocryxz4ts2wtsi1bawtjndt', 'hkxsbgefntzwsodvj3ztvp3gm1',
 'j7dlvmya0aazq8muvzafu1ezmo', 'wbg4irhu8cts7vqtzmx6f8xwg7',
 'angticjoeipyve4gpwgwd7nv53', 'd27f4w0n0sys7hpxdedthm0q1s',
 ...
 'ri3lqgshwnyc7tqb1ray1mvllb', '9nla4hgs9ofnyltgwookj3vm2x',
 'wsunenfdtesnbkfiqjhahhsofp', 'thsfr16jtgrj9bdvczskbxftdf',
 'kdepdh8lsiroc3uxvejhsjivcs', 'yzddjkp8wkrghj6ni6x9qxiwgd',
 '45lbumns5lqxr9x0ajdxdaz7vl', 'dwd3dybliaxryowwsadkmliluw',
 'asyeuoax8uaw3np3tlyxpcfxzm', 'wfsihoehobxkipt3731virtjqq']
Length: 231717, dtype: string

In [91]:
col_to_check = 'createdAt'
safeload_orders_df[col_to_check].unique()

<StringArray>
['2024-07-01', '2024-07-02', '2024-07-03', '2024-07-04', '2024-07-05',
 '2024-07-06', '2024-07-07', '2024-07-08', '2024-07-09', '2024-07-10',
 ...
 '2022-07-10', '2022-07-09', '2022-07-08', '2022-07-07', '2022-07-06',
 '2022-07-05', '2022-07-04', '2022-07-03', '2022-07-02', '2022-07-01']
Length: 957, dtype: string

In [92]:
col_to_check = 'createdAtTime'
safeload_orders_df[col_to_check].unique()

<StringArray>
['2024-07-01 19:41:44', '2024-07-01 18:34:12', '2024-07-01 16:08:26',
 '2024-07-01 16:02:11', '2024-07-01 15:46:12', '2024-07-01 09:01:16',
 '2024-07-01 08:56:34', '2024-07-01 08:45:56', '2024-07-01 08:39:48',
 '2024-07-01 08:33:35',
 ...
 '2022-07-01 18:56:40', '2022-07-01 18:22:27', '2022-07-01 16:12:06',
 '2022-07-01 14:08:23', '2022-07-01 12:43:26', '2022-07-01 11:41:06',
 '2022-07-01 09:52:34', '2022-07-01 09:06:21', '2022-07-01 07:27:10',
 '2022-07-01 06:36:05']
Length: 536393, dtype: string

In [93]:
col_to_check = 'paymentGateway'
safeload_orders_df[col_to_check].unique()

<StringArray>
['stripe', 'discount', 'welfare', 'scalapay', 'coupon', 'satispay']
Length: 6, dtype: string

In [94]:
col_to_check = 'paymentBrand'
safeload_orders_df[col_to_check].unique()

<StringArray>
['visa', '', 'mastercard', 'amex', 'diners', 'discover', 'unionpay']
Length: 7, dtype: string

In [95]:
col_to_check = 'pickup'
safeload_orders_df[col_to_check].unique()

<StringArray>
['False', 'True']
Length: 2, dtype: string

In [96]:
col_to_check = 'pickupComplete'
safeload_orders_df[col_to_check].unique()

<StringArray>
['False', 'True']
Length: 2, dtype: string

In [97]:
col_to_check = 'source'
safeload_orders_df[col_to_check].unique()

<StringArray>
['website', 'app', '']
Length: 3, dtype: string

In [98]:
col_to_check = 'tenant'
safeload_orders_df[col_to_check].unique()

<StringArray>
[ 'snowit_esterno',         'tribala',       'discovera',          'snowit',
  'bikeit_esterno',          'bikeit',  'snowit_welfare',  'bikeit_welfare',
 'tribala_esterno', 'tribala_welfare',                '']
Length: 11, dtype: string

In [99]:
col_to_check = 'paymentAttempts'
safeload_orders_df[col_to_check].unique()

<StringArray>
[ '2',  '1',  '3',  '4',  '5', '11',  '6',  '7', '10',  '8',  '9', '13', '22',
 '12', '16', '14', '19', '18', '17', '26', '21', '15', '25', '34', '23', '24',
 '31']
Length: 27, dtype: string

In [100]:
col_to_check = 'timeZone'
safeload_orders_df[col_to_check].unique()

<StringArray>
['', 'Europe/Rome']
Length: 2, dtype: string

In [101]:
col_to_check = 'clientInfo'
safeload_orders_df[col_to_check].unique()

<StringArray>
[                                           '{"browser":{"major":"126","name":"Chrome","version":"126.0.0.0"},"cpu":{"architecture":"amd64"},"device":{"model":null,"type":null,"vendor":null},"engine":{"name":"Blink","version":"126.0.0.0"},"os":{"name":"Windows","version":"10"},"ua":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36"}',
 '{"browser":{"major":"126","name":"Chrome","version":"126.0.6478.153"},"cpu":{"architecture":null},"device":{"model":"iPhone","type":"mobile","vendor":"Apple"},"engine":{"name":"WebKit","version":"605.1.15"},"os":{"name":"iOS","version":"17.5"},"ua":"Mozilla/5.0 (iPhone; CPU iPhone OS 17_5 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) CriOS/126.0.6478.153 Mobile/15E148 Safari/604.1"}',
                           '{"browser":{"major":"126","name":"Chrome","version":"126.0.0.0"},"cpu":{"architecture":null},"device":{"model":"Macintosh","type":null,"vendor":"Apple"},"engi

In [102]:
# Read and fix
import numpy as np

orders_df = pd.read_csv(orders_in_path,
                       keep_default_na=False,
                       na_values=NA_VALUES, # assign the ones previously defined
                       dtype={
                           'user.uid': 'string',
                           'order.uid': 'string',
                           'createdAt': 'string',
                           'createdAtTime': 'string',
                           'paymentGateway': 'string',
                           'paymentBrand': 'string',
                           'pickup': 'boolean',
                           'pickupComplete': 'boolean',
                           'source': 'string',
                           'tenant': 'string',
                           'paymentAttempts': 'Int64',
                           'timeZone': 'string',
                           'clientInfo': 'string',
                       }
                       )

orders_df['createdAt'] = pd.to_datetime(orders_df['createdAt'])

orders_df['createdAtTime'] = pd.to_datetime(orders_df['createdAtTime'])

In [103]:
# CHECK Primary Key VALIDITY

# SELECT count(1) as num_rows
# FROM orders_df
# WHERE orders.uid IS NULL

display(
    orders_df
    .loc[lambda tbl: tbl['order.uid'].isnull()]
    .assign(aux=1.0)
    .shape[0]
)

# SELECT order.uid, count(1) as num_rows
# FROM orders_df
# GROUP BY order.uid
# HAVING num_rows > 1

display(
    orders_df
    .assign(aux=1.0)
    .groupby(['order.uid'], dropna=False)
    .agg(num_rows=('aux', pd.Series.count))
    .loc[lambda tbl: tbl['num_rows'] > 1]
)

0

,num_rows
order.uid,


In [104]:
# CHECK Foreign Key VALIDITY

# SELECT
#   A.user.uid,
#   count(1) as num_rows
# FROM 
#   (SELECT DISTINCT user.uid
#   FROM orders_df
#   WHERE user.uid IS NOT NULL) AS A
#   LEFT JOIN
#   (SELECT user.uid, 1.0 AS in_users
#   FROM users_df) AS B
#   ON A.user.uid = B.user.uid
# GROUP BY in_users
# HAVING num_rows > 1

(orders_df
 [['user.uid']]
 .loc[lambda tbl: tbl['user.uid'].notnull()]
 .drop_duplicates()
 .merge(users_df[['user.uid']].assign(in_users=1.0),
        how='left',
        on='user.uid'
        )
 .assign(aux=1.0)
 .groupby(['in_users'], dropna=False)
 .agg(num_rows=('aux', pd.Series.count))
 .loc[lambda tbl: tbl['num_rows'] > 1]
)

,num_rows
in_users,
1.0,161266
NaN,70451


In [105]:
# issue, I have 70451 orders with an empty left join with the user.uid table
# let's visualize these 70451 profiles

In [106]:
# SELECT
#   A.user.uid,
#   count(1) as num_rows
# FROM 
#   (SELECT DISTINCT user.uid
#   FROM orders_df
#   WHERE user.uid IS NOT NULL) AS A
#   LEFT JOIN
#   (SELECT user.uid, 1.0 AS in_users
#   FROM users_df) AS B
#   ON A.user.uid = B.user.uid
# WHERE
#   in_users IS NULL

(orders_df
 [['user.uid']]
 .loc[lambda tbl: tbl['user.uid'].notnull()]
 .drop_duplicates()
 .merge(users_df[['user.uid']].assign(in_users=1.0),
        how='left',
        on='user.uid'
        )
 .loc[lambda tbl: tbl['in_users'].isnull()]
)

,user.uid,in_users
1,ghol7lwjlhewwvasejlxa9fv00,NaN
2,nhuh3scqoktj6n5mnqkvgyh43h,NaN
3,oyekz0wyxcplvrpencpuue6sdt,NaN
4,1p0ocryxz4ts2wtsi1bawtjndt,NaN
11,ox5rgm93oezmrqao7lbg05ovyk,NaN
...,...,...
231694,aouwuww0svnywu5f85gmbnbzb5,NaN
231702,jhx8gn3nqje3ztcch1or4yjpnb,NaN
231704,wnnh4uxef0m2il1kys1xbogkry,NaN
231707,ri3lqgshwnyc7tqb1ray1mvllb,NaN


In [107]:
# from the table printed before I select just the column ['user.uid'],
# those IDs without a correspondence in the users_df
order_fail_useruids = (orders_df
    [['user.uid']]
    .loc[lambda tbl: tbl['user.uid'].notnull()]
    .drop_duplicates()
    .merge(users_df[['user.uid']].assign(in_users=1.0),
            how='left',
            on='user.uid'
            )
    .loc[lambda tbl: tbl['in_users'].isnull()]
    ['user.uid']
)
display(order_fail_useruids)

# extract now the profiles with "inconsistent" 'user.uid'
order_fail_useruid_df = orders_df.loc[lambda tbl: tbl['user.uid'].isin(order_fail_useruids)]
display(order_fail_useruid_df)

1         ghol7lwjlhewwvasejlxa9fv00
2         nhuh3scqoktj6n5mnqkvgyh43h
3         oyekz0wyxcplvrpencpuue6sdt
4         1p0ocryxz4ts2wtsi1bawtjndt
11        ox5rgm93oezmrqao7lbg05ovyk
                     ...            
231694    aouwuww0svnywu5f85gmbnbzb5
231702    jhx8gn3nqje3ztcch1or4yjpnb
231704    wnnh4uxef0m2il1kys1xbogkry
231707    ri3lqgshwnyc7tqb1ray1mvllb
231712    yzddjkp8wkrghj6ni6x9qxiwgd
Name: user.uid, Length: 70451, dtype: string

,order.uid,user.uid,createdAt,createdAtTime,paymentGateway,paymentBrand,pickup,pickupComplete,source,tenant,paymentAttempts,timeZone,clientInfo
1,termljwsr2gecnzwks,ghol7lwjlhewwvasejlxa9fv00,2024-07-01,2024-07-01 18:34:12,discount,<NA>,False,False,website,snowit_esterno,1,<NA>,"{""browser"":{""major"":""126"",""name"":""Chrome"",""ver..."
2,ko8axscf5grobnz9in,nhuh3scqoktj6n5mnqkvgyh43h,2024-07-01,2024-07-01 16:08:26,discount,<NA>,False,False,website,snowit_esterno,1,<NA>,"{""browser"":{""major"":""126"",""name"":""Chrome"",""ver..."
3,tu8b9haokkimu72aza,oyekz0wyxcplvrpencpuue6sdt,2024-07-01,2024-07-01 16:02:11,stripe,mastercard,False,False,website,snowit_esterno,2,<NA>,"{""browser"":{""major"":""126"",""name"":""Chrome"",""ver..."
4,a8mvwuv6ileh9aqblu,1p0ocryxz4ts2wtsi1bawtjndt,2024-07-01,2024-07-01 15:46:12,stripe,<NA>,False,False,website,tribala,2,<NA>,"{""browser"":{""major"":""125"",""name"":""Chrome"",""ver..."
5,kmwabdpnhtm46p8me8,nhuh3scqoktj6n5mnqkvgyh43h,2024-07-01,2024-07-01 09:01:16,discount,<NA>,False,False,website,snowit_esterno,1,<NA>,"{""browser"":{""major"":""126"",""name"":""Chrome"",""ver..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
549883,bjzuut5lvj76ad6jbp,wnnh4uxef0m2il1kys1xbogkry,2022-07-01,2022-07-01 21:02:41,discount,<NA>,False,False,website,snowit_esterno,1,Europe/Rome,"{""browser"":{""major"":""102"",""name"":""Chrome"",""ver..."
549884,ur0t4p0hejetoesg3h,wnnh4uxef0m2il1kys1xbogkry,2022-07-01,2022-07-01 20:56:39,discount,<NA>,False,False,website,snowit_esterno,1,Europe/Rome,"{""browser"":{""major"":""102"",""name"":""Chrome"",""ver..."
549886,3unzmjqzzmwgnbil56,iz4qy0qs0qre4g8ndczecyms3s,2022-07-01,2022-07-01 19:51:24,stripe,<NA>,False,False,website,snowit_esterno,1,Europe/Rome,"{""browser"":{""major"":""103"",""name"":""Chrome"",""ver..."
549888,mhosjzeznativvqfnz,ri3lqgshwnyc7tqb1ray1mvllb,2022-07-01,2022-07-01 19:21:13,satispay,<NA>,False,False,website,snowit_esterno,1,Europe/Rome,"{""browser"":{""major"":""15"",""name"":""Mobile Safari..."


In [108]:
# FIX Foreing Key ISSUE: remove them ! 
orders_df.drop(list(order_fail_useruid_df.index), inplace=True)

display(
    orders_df
    [['user.uid']]
    .loc[lambda tbl: tbl['user.uid'].notnull()]
    .drop_duplicates()
    .merge(users_df[['user.uid']].assign(in_users=1.0),
            how='left',
            on='user.uid'
            )
    .assign(aux=1.0)
    .groupby(['in_users'], dropna=False)
    .agg(num_rows=('aux', pd.Series.count))
    .loc[lambda tbl: tbl['num_rows'] > 1]
)
# now, as expected, I don't have null values after the left join

,num_rows
in_users,
1.0,161266


## Load Order Details

In [109]:
safeload_order_details_df = pd.read_csv(order_details_in_path,
                                        dtype='string',
                                        na_values=[],
                                        keep_default_na=False
                                        )

In [110]:
safeload_order_details_df.columns

Index(['item.uid', 'order.uid', 'item.date', 'product.uid',
       'product.dynamicPricing', 'item.amount', 'item.discount',
       'product.type', 'item.zoneName', 'product.durationHours',
       'item.profiles', 'item.variantName', 'item.slotName',
       'item.snowitcardNumber', 'item.status'],
      dtype='object')

In [111]:
safeload_order_details_df.shape

(1420603, 15)

In [112]:
col_to_check = 'item.uid'
safeload_order_details_df[col_to_check].unique()
# it's reasonable that the dimension is lower than the 'shape' since the same item
# can be associated with more orders

<StringArray>
['rWzvRD0hXOShtjTzx5Ca', 'mmWAugpNnPVHRcePWiiH', 'bShIBhXedPkDTmZpmczI',
 'axrCVxzH7eVW2EbaKHho', '4Neyeg0izzwue8RbJrPH', 'Wf4gfrRUh3ZDYp15NNql',
 'sSPT3hVthnvf2GDryYaD', 'CSbMX1gUCNaU6tx7aPxu', '2RwFbVS1BRIchiqgDQQ1',
 'QihrElx8hg0HK8eKxXL6',
 ...
 '6e6q400crcOwUnyGSGcu', 'oEE4Z1zF7SK4vm0PrMrW', '6YbL5fFmf6QwagnSSEYa',
 'xFgt4Cqaa8Z4nevGOdjD', 'tge44Z5KbElSymEoXQ09', '9ka9Ogu0nyxEkEwKMaNT',
 'xWAoWDvpz1JilvtElDm9', 'Cdhlra1fhDx4vr0xwCeJ', '2Ghwkit8jdMI6HgwutI5',
 'Xf9Njxh1jJNDpBqd7UxV']
Length: 1403655, dtype: string

In [113]:
col_to_check = 'order.uid'
safeload_order_details_df[col_to_check].unique()
# the same order can have more items

<StringArray>
['4y9zqqvldfqr9n2xnu', 'termljwsr2gecnzwks', 'ko8axscf5grobnz9in',
 'tu8b9haokkimu72aza', 'a8mvwuv6ileh9aqblu', 'kmwabdpnhtm46p8me8',
 'i9eovtgp3uxbmesebv', 'nsc55zfspubshggcm5', 'ltr5f48gwicqwcnzkf',
 'eyx8wesrjaeltlgqna',
 ...
 '5f8he6aq0sbnmrrwzq', 'g0tkwbe686jnynbxs9', 'aoud70hexrj3uryxmd',
 'h0vb0rmb2mk4c7s3tv', 'v1obzy9nwp99nanhue', '6icnjfrltzhdi2eato',
 '4n6z8api7q0cz8jxox', 'pdubfidjsudk9mhwdc', 'f8zj28hdatvraibekk',
 'wy69h8jsgozdrxlu9i']
Length: 549900, dtype: string

In [114]:
col_to_check = 'item.status'
safeload_order_details_df[col_to_check].unique()

<StringArray>
['fulfilled', 'canceled', 'transfer', 'ok', 'processing', 'on-hold']
Length: 6, dtype: string

In [115]:
# verify that each triplet (item.uid, order.uid, item.status) is unique

# length
total_rows = len(safeload_order_details_df)

# unique rows with respect to the three columns
unique_rows = safeload_order_details_df.drop_duplicates(subset=['item.uid', 'order.uid', 'item.status']).shape[0]

# check if correspondence
if total_rows == unique_rows:
    print("Dataset is consistent: each triplet is unique")
else:
    print(f"Found {total_rows - unique_rows} duplicates")


Dataset is consistent: each triplet is unique


In [116]:
col_to_check = 'item.date'
safeload_order_details_df[col_to_check].unique()

<StringArray>
['2024-07-05', '2024-12-27', '2024-07-02', '2024-08-30', '2024-07-26',
 '2024-07-01', '2024-07-03', '2024-08-11', '2024-07-04', '2024-10-15',
 ...
 '2022-07-11', '2022-07-10', '2022-07-09', '2022-07-08', '2022-07-07',
 '2022-07-06', '2022-07-05', '2022-07-04', '2022-07-03', '2022-07-02']
Length: 979, dtype: string

In [117]:
col_to_check = 'product.uid'
safeload_order_details_df[col_to_check].unique()

<StringArray>
['OwhliJmMeSCILiMFiKTp',  'form:PO9LFLX23J2024', '9oNSf7M6J7BNWakW2otU',
 'Kh1RLBt3sC5Z6amhRuBn', 'Tv7I6hDcWmu5DWBxN24X', 'we7MOniCQlC4vAgFxE27',
 'DtB2im6mhHILukxObBpO', 'nOiVTAEG48YK62ycSCdu', 'z1g2LhWX3b9VLD57nqUi',
 '9IeTI9YanEx3NKDJpmbJ',
 ...
 '6wu1Xu8TYTHnA3UXjeBu', 'wVf8LHxrSumU7aqpEgEl', 'gw2junAn94xtUbd18iHq',
 'G7WCPsjFcRHtkq6lmfEk', 'wONr18elM8vnXBA1js7E', 'qK6GXLOAZHLM2nUurTsv',
 '8uLIkozKS1UdHd2Z1XlV', '9TYrrypnqRvNRhHhLLKk', 'F5n33TDKva6A0D5kC1eP',
 'iL4iqfl1EwSlixe3nJd6']
Length: 9928, dtype: string

In [118]:
col_to_check = 'product.dynamicPricing'
safeload_order_details_df[col_to_check].unique()

<StringArray>
['False', 'True']
Length: 2, dtype: string

In [119]:
col_to_check = 'item.amount'
safeload_order_details_df[col_to_check].unique()

# since prices are always positive, negative numbers here shown are considered to be bad formatted ones.

<StringArray>
[   '59.5',     '0.0',    '26.0',   '-26.0',   '122.5',    '60.0',     '1.0',
    '-1.0',    '10.0',    '50.0',
 ...
   '22.99',  '1126.0', '-1126.0',  '832.85', '-832.85',  '118.01',   '53.62',
   '76.62',   '69.22',  '2900.0']
Length: 6266, dtype: string

In [120]:
sorted_values = np.sort(safeload_order_details_df[col_to_check].unique())
sorted_values[:500]

array(['-0.01', '-1.0', '-1.5', '-10.0', '-10.4', '-10.5', '-100.0',
       '-100.12', '-100.25', '-100.5', '-100.8', '-1000.0', '-1004.0',
       '-101.0', '-101.15', '-101.25', '-101.7', '-102.0', '-102.5',
       '-102.6', '-102.7', '-102.8', '-103.0', '-103.3', '-103.5',
       '-103.55', '-103.68', '-103.7', '-103.74', '-103.8', '-103.95',
       '-1030.0', '-104.0', '-104.6', '-104.85', '-1040.0', '-1045.0',
       '-105.0', '-105.2', '-105.44', '-105.5', '-1058.0', '-106.0',
       '-106.4', '-106.8', '-107.0', '-107.1', '-107.35', '-107.75',
       '-107.95', '-108.0', '-108.3', '-109.0', '-109.4', '-109.5',
       '-1090.0', '-11.0', '-11.1', '-11.5', '-11.7', '-110.0', '-110.2',
       '-110.3', '-110.5', '-110.7', '-110.75', '-110.8', '-11086.0',
       '-111.0', '-111.2', '-111.5', '-111.75', '-111.8', '-112.0',
       '-112.2', '-112.5', '-1126.0', '-113.0', '-113.35', '-113.75',
       '-114.0', '-114.3', '-114.75', '-115.0', '-115.5', '-115.6',
       '-1150.0', '-116.0'

In [121]:
safeload_order_details_df[col_to_check] = (
    safeload_order_details_df[col_to_check]
    .astype(float)          
    .abs()                  
)

sorted_values_abs = np.sort(safeload_order_details_df[col_to_check].unique())
sorted_values_abs[:500]

array([0.000e+00, 1.000e-02, 5.000e-01, 9.500e-01, 1.000e+00, 1.500e+00,
       2.000e+00, 2.010e+00, 2.100e+00, 2.500e+00, 2.750e+00, 3.000e+00,
       3.500e+00, 4.000e+00, 4.500e+00, 5.000e+00, 5.100e+00, 5.220e+00,
       5.500e+00, 5.950e+00, 6.000e+00, 6.400e+00, 6.500e+00, 7.000e+00,
       7.100e+00, 7.200e+00, 7.250e+00, 7.500e+00, 7.650e+00, 7.750e+00,
       7.900e+00, 8.000e+00, 8.100e+00, 8.400e+00, 8.500e+00, 8.550e+00,
       8.800e+00, 9.000e+00, 9.200e+00, 9.400e+00, 9.500e+00, 9.600e+00,
       9.900e+00, 1.000e+01, 1.020e+01, 1.040e+01, 1.050e+01, 1.060e+01,
       1.080e+01, 1.091e+01, 1.100e+01, 1.105e+01, 1.110e+01, 1.120e+01,
       1.140e+01, 1.150e+01, 1.170e+01, 1.190e+01, 1.200e+01, 1.235e+01,
       1.250e+01, 1.260e+01, 1.275e+01, 1.280e+01, 1.300e+01, 1.325e+01,
       1.330e+01, 1.350e+01, 1.360e+01, 1.380e+01, 1.400e+01, 1.425e+01,
       1.440e+01, 1.445e+01, 1.450e+01, 1.470e+01, 1.475e+01, 1.485e+01,
       1.500e+01, 1.520e+01, 1.525e+01, 1.530e+01, 

In [122]:
col_to_check = 'item.discount'
safeload_order_details_df[col_to_check].unique()

<StringArray>
['True', 'False']
Length: 2, dtype: string

In [123]:
col_to_check = 'product.type'
safeload_order_details_df[col_to_check].unique()

<StringArray>
[                'rental~bike',                'hotel~coupon',
                        'food',                  'experience',
                'bundle~train',                   'transport',
                     'service',                     'skipass',
                    'giftcard',              'skipass~return',
                   'itinerary',                   'insurance',
               'bundle~travel',                      'lesson',
                  'snowitcard',               'skipass~hotel',
                       'hotel',                      'bundle',
                      'rental',             'transport~ferry',
                  'bundle~tab',                      'coupon',
          'skipass~membership',                    'seasonal',
           'experience~museum', 'skipass~membership-seasonal',
             'voucher~skipass',             'skipass~dynamic',
              'bundle~dynamic',                     'parking']
Length: 30, dtype: string

In [124]:
col_to_check = 'item.zoneName'
safeload_order_details_df[col_to_check].unique()

<StringArray>
[                       'Bormio',           'Pontedilegno-Tonale',
                  'Bardonecchia',                        'Italia',
           'Desenzano del Garda',                 'Lago Maggiore',
                  'Lago di Como',                              '',
                   "Lago d'Iseo",      'Barzio - Piani di Bobbio',
 ...
          'Gressoney-La-Trinité',                         'Badia',
                      'Dobbiaco',                      'Tarvisio',
                     'Sestriere',                  "Sauze D'Oulx",
 'Valmalenco Bernina Ski Resort',           'Moggio - Artavaggio',
                     'Schärding',                        'Snowit']
Length: 151, dtype: string

In [125]:
col_to_check = 'product.durationHours'
safeload_order_details_df[col_to_check].unique()

<StringArray>
['12.0', '', '4.0', '3.0', '1.0', '2.0', '5.0', '6.0', '8.0', '7.0']
Length: 10, dtype: string

In [126]:
col_to_check = 'item.profiles'
safeload_order_details_df[col_to_check].head(60)

0     uhozpzih6zz0gb8vsnrl
1                         
2                         
3                         
4                         
5                         
6                         
7                         
8                         
9                         
10    8xhyvkzruc1g4d8zenik
11                        
12                        
13                        
14                        
15                        
16                        
17                        
18                        
19                      _1
20                      _1
21                        
22                        
23                      _1
24                      _1
25                      _1
26                      _1
27                      _1
28                      _1
29                      _1
30                        
31                        
32                        
33    i96cld933twu3lpvuowy
34    ddxdlztbtpzjllq8eykp
35                        
36                        
3

In [127]:
col_to_check = 'item.variantName'
safeload_order_details_df[col_to_check].unique()

<StringArray>
[                    'Rossignol Mandate Shift',
                                            '',
                   'Bottecchia 8avio Ultimate',
                          '0.4583333333333333',
                                      'Adulto',
                         'Ragazzo (4-13 anni)',
                    'Bottecchia BE38 Avok 900',
               'Viaggio di 9 giorni e 8 notti',
                         'Lezioni di kitesurf',
                                        'Uomo',
 ...
 'Overvolt AM 5.6 2022 Pomeriggio 14:00-18:00',
                               'all track 27+',
                  'Double-Triple Room Economy',
             'Carnet degustazione - 7 assaggi',
             'Carnet degustazione - 3 assaggi',
                                    '1 adulto',
                          'Pacchetto famiglia',
      'Carnet degustazione - 7 assaggi + Tast',
                                  '9:00-12:00',
     'Lapierre Overvolt AM 5.5 Mezza Giornata']
Length: 927, dtype: s

In [128]:
col_to_check = 'item.slotName'
safeload_order_details_df[col_to_check].unique()

<StringArray>
[                                   'L',
                                     '',
                                   '58',
                                  'LXL',
                                    'M',
                           'Intermedio',
              'Posto in camerata mista',
                                   '53',
                                    'S',
               'Posto in camera doppia',
 ...
 'Entrance from 13:00 to 15:00 (Woman)',
  'Entrance from 9:00 to 11:00 (Woman)',
                          '14:30-16:30',
                   'Trasporto autonomo',
                          '14:00-16:30',
                   'Partenza da Torino',
                  'Partenza da Bergamo',
                          '19:30-23:30',
                                   'XS',
                           '9:00-11:30']
Length: 276, dtype: string

In [129]:
col_to_check = 'item.snowitcardNumber'
safeload_order_details_df[col_to_check].unique()

<StringArray>
[         '', '4586597.0', '4586494.0', '4586520.0', '4586601.0', '4586438.0',
 '4586449.0', '4303271.0', '4303318.0', '6486093.0',
 ...
 '4789527.0', '7443947.0', '7443958.0', '4743417.0', '6357876.0', '6357072.0',
 '6357061.0',    '4145.0', '4687472.0', '1796685.0']
Length: 221230, dtype: string

In [130]:
# Read and fix
import numpy as np

order_details_df = pd.read_csv(order_details_in_path,
                       keep_default_na=False,
                       na_values=NA_VALUES, # assign the ones previously defined
                       dtype={
                           'item.uid': 'string',
                           'order.uid': 'string',
                           'item.status': 'string',
                           'item.date': 'string',
                           'product.uid': 'string',
                           'product.dynamicPricing': 'boolean',
                           'item.amount': 'Float32',
                           'item.discount': 'boolean',
                           'product.type': 'string',
                           'item.zoneName': 'string',
                           'product.durationHours': 'Float32',
                           'item.profiles': 'string',
                           'item.variantName': 'string',
                           'item.slotName': 'string',
                           'item.snowitcardNumber': 'string',
                       }
                       )

order_details_df['item.date'] = pd.to_datetime(order_details_df['item.date'])

order_details_df['item.amount'] = (order_details_df['item.amount'].abs())

In [131]:
# CHECK Foreign Key VALIDITY

# SELECT
#   A.order.uid,
#   count(1) as num_rows
# FROM 
#   (SELECT DISTINCT order.uid
#   FROM order_details_df
#   WHERE order.uid IS NOT NULL) AS A
#   LEFT JOIN
#   (SELECT order.uid, 1.0 AS in_orders
#   FROM orders_df) AS B
#   ON A.order.uid = B.order.uid
# GROUP BY in_orders
# HAVING num_rows > 1

(order_details_df
 [['order.uid']]
 .loc[lambda tbl: tbl['order.uid'].notnull()]
 .drop_duplicates()
 .merge(orders_df[['order.uid']].assign(in_orders=1.0),
        how='left',
        on='order.uid'
        )
 .assign(aux=1.0)
 .groupby(['in_orders'], dropna=False)
 .agg(num_rows=('aux', pd.Series.count))
 .loc[lambda tbl: tbl['num_rows'] > 1]
)

,num_rows
in_orders,
1.0,385168
NaN,164732


In [132]:
# issue, I have 164732 detailed orders with an empty left join with the orders table
# let's visualize these profiles

In [133]:
# SELECT
#   A.order.uid,
#   count(1) as num_rows
# FROM 
#   (SELECT DISTINCT order.uid
#   FROM order_details_df
#   WHERE order.uid IS NOT NULL) AS A
#   LEFT JOIN
#   (SELECT order.uid, 1.0 AS in_orders
#   FROM orders_df) AS B
#   ON A.order.uid = B.order.uid
# WHERE
#   in_orders IS NULL

(order_details_df
 [['order.uid']]
 .loc[lambda tbl: tbl['order.uid'].notnull()]
 .drop_duplicates()
 .merge(orders_df[['order.uid']].assign(in_orders=1.0),
        how='left',
        on='order.uid'
        )
 .loc[lambda tbl: tbl['in_orders'].isnull()]
)

,order.uid,in_orders
1,termljwsr2gecnzwks,NaN
2,ko8axscf5grobnz9in,NaN
3,tu8b9haokkimu72aza,NaN
4,a8mvwuv6ileh9aqblu,NaN
5,kmwabdpnhtm46p8me8,NaN
...,...,...
549883,bjzuut5lvj76ad6jbp,NaN
549884,ur0t4p0hejetoesg3h,NaN
549886,3unzmjqzzmwgnbil56,NaN
549888,mhosjzeznativvqfnz,NaN


In [134]:
# from the table printed before I select just the column ['order.uid'],
# those IDs without a correspondence in the orders_df

order_fail_useruids = (order_details_df
    [['order.uid']]
    .loc[lambda tbl: tbl['order.uid'].notnull()]
    .drop_duplicates()
    .merge(orders_df[['order.uid']].assign(in_orders=1.0),
            how='left',
            on='order.uid'
            )
    .loc[lambda tbl: tbl['in_orders'].isnull()]
    ['order.uid']
)
display(order_fail_useruids)

# extract now the profiles with "inconsistent" 'user.uid'
order_fail_useruid_df = order_details_df.loc[lambda tbl: tbl['order.uid'].isin(order_fail_useruids)]
display(order_fail_useruid_df)

1         termljwsr2gecnzwks
2         ko8axscf5grobnz9in
3         tu8b9haokkimu72aza
4         a8mvwuv6ileh9aqblu
5         kmwabdpnhtm46p8me8
                 ...        
549883    bjzuut5lvj76ad6jbp
549884    ur0t4p0hejetoesg3h
549886    3unzmjqzzmwgnbil56
549888    mhosjzeznativvqfnz
549894    v1obzy9nwp99nanhue
Name: order.uid, Length: 164732, dtype: string

,item.uid,order.uid,item.date,product.uid,product.dynamicPricing,item.amount,item.discount,product.type,item.zoneName,product.durationHours,item.profiles,item.variantName,item.slotName,item.snowitcardNumber,item.status
1,mmWAugpNnPVHRcePWiiH,termljwsr2gecnzwks,2024-12-27,form:PO9LFLX23J2024,False,0.0,False,hotel~coupon,Pontedilegno-Tonale,<NA>,<NA>,<NA>,<NA>,<NA>,fulfilled
2,bShIBhXedPkDTmZpmczI,ko8axscf5grobnz9in,2024-07-02,9oNSf7M6J7BNWakW2otU,False,26.0,False,food,Bardonecchia,4.0,<NA>,<NA>,<NA>,<NA>,canceled
3,axrCVxzH7eVW2EbaKHho,ko8axscf5grobnz9in,2024-07-02,9oNSf7M6J7BNWakW2otU,False,26.0,False,food,Bardonecchia,4.0,<NA>,<NA>,<NA>,<NA>,canceled
4,axrCVxzH7eVW2EbaKHho,ko8axscf5grobnz9in,2024-07-02,9oNSf7M6J7BNWakW2otU,False,26.0,False,food,Bardonecchia,4.0,<NA>,<NA>,<NA>,<NA>,transfer
5,4Neyeg0izzwue8RbJrPH,ko8axscf5grobnz9in,2024-07-02,9oNSf7M6J7BNWakW2otU,False,26.0,False,food,Bardonecchia,4.0,<NA>,<NA>,<NA>,<NA>,canceled
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1420574,0NGCJGZ3NyfzdL5cXRI5,mhosjzeznativvqfnz,2022-07-03,1Ms1ktHDULulVrLvnCI3,False,5.0,False,skipass,Barzio - Piani di Bobbio,12.0,<NA>,<NA>,<NA>,<NA>,fulfilled
1420586,X2S1z1uv9T03UsDC4M53,v1obzy9nwp99nanhue,2022-07-27,30gOXuGs811l6EBBA83w,False,0.0,False,experience,Livigno,12.0,<NA>,<NA>,<NA>,<NA>,transfer
1420587,UfwVcrM34CNwe7VBGfjY,v1obzy9nwp99nanhue,2022-07-27,30gOXuGs811l6EBBA83w,False,0.0,False,experience,Livigno,12.0,<NA>,<NA>,<NA>,<NA>,canceled
1420588,UfwVcrM34CNwe7VBGfjY,v1obzy9nwp99nanhue,2022-07-27,30gOXuGs811l6EBBA83w,False,0.0,False,experience,Livigno,12.0,<NA>,<NA>,<NA>,<NA>,transfer


In [135]:
# FIX Foreing Key ISSUE: remove them ! 
order_details_df.drop(list(order_fail_useruid_df.index), inplace=True)

display(
    order_details_df
    [['order.uid']]
    .loc[lambda tbl: tbl['order.uid'].notnull()]
    .drop_duplicates()
    .merge(orders_df[['order.uid']].assign(in_orders=1.0),
            how='left',
            on='order.uid'
            )
    .assign(aux=1.0)
    .groupby(['in_orders'], dropna=False)
    .agg(num_rows=('aux', pd.Series.count))
    .loc[lambda tbl: tbl['num_rows'] > 1]
)
# now, as expected, I don't have null values after the left join

,num_rows
in_orders,
1.0,385168


In [136]:
# CHECK Profiles VALIDITY

# SELECT
#   A.item.profiles,
#   count(1) as num_rows
# FROM 
#   (SELECT DISTINCT item.profiles
#   FROM order_details_df
#   WHERE item.profiles IS NOT NULL) AS A
#   LEFT JOIN
#   (SELECT profile.uid, 1.0 AS in_profiles
#   FROM profiles_df) AS B
#   ON A.item.profiles = B.profile.uid
# GROUP BY in_profiles
# HAVING num_rows > 1

(order_details_df
 [['item.profiles']]
 .loc[lambda tbl: tbl['item.profiles'].notnull()]
 .drop_duplicates()
 .merge(profiles_df[['profile.uid']].assign(in_profiles=1.0),
        how='left',
        left_on='item.profiles',
        right_on='profile.uid'
        )
 .assign(aux=1.0)
 .groupby(['in_profiles'], dropna=False)
 .agg(num_rows=('aux', pd.Series.count))
 .loc[lambda tbl: tbl['num_rows'] > 1]
)

,num_rows
in_profiles,
1.0,20189
NaN,374


In [137]:
# issue, I have 374 detailed orders with an empty left join with the profiles table
# let's visualize these profiles

In [138]:
# SELECT
#   A.item.profiles,
#   count(1) as num_rows
# FROM 
#   (SELECT DISTINCT  item.profiles
#   FROM order_details_df
#   WHERE item.profles IS NOT NULL) AS A
#   LEFT JOIN
#   (SELECT profile.uid, 1.0 AS in_profiles
#   FROM profiles_df) AS B
#   ON A.item.profiles = B.profile.uid
# WHERE
#   in_profiles IS NULL

(order_details_df
 [['item.profiles']]
 .loc[lambda tbl: tbl['item.profiles'].notnull()]
 .drop_duplicates()
 .merge(profiles_df[['profile.uid']].assign(in_profiles=1.0),
        how='left',
        left_on='item.profiles',
        right_on='profile.uid'
        )
 .loc[lambda tbl: tbl['in_profiles'].isnull()]
)

,item.profiles,profile.uid,in_profiles
1,_1,<NA>,NaN
5,_1;_2,<NA>,NaN
85,_1;_2;_3;_4,<NA>,NaN
97,muyejxsmorfojxxg1hlb;zkrsoiswdwunbosphcav,<NA>,NaN
109,_1;_2;_3,<NA>,NaN
...,...,...,...
20540,avrn9k3o2mu2fjy9acsh,<NA>,NaN
20541,wo45fqxde6igpavfqj2a,<NA>,NaN
20542,5ojsyocyu4nfqocvcim1,<NA>,NaN
20555,g5fxgajus6hxnouuqxv7;lwpc3d8rufleyqqpopqg,<NA>,NaN


In [139]:
# for the time being, I decide to still keep these values since they could have information about sold products, even if not linked to a profile.

## Load Reviews

In [140]:
REVIEWS_IN_FILENAME = 'reviews.csv'
REVIEWS_PKL_FILENAME = 'reviews.pkl'

reviews_in_path = data_in_dir_path.joinpath(REVIEWS_IN_FILENAME) 
reviews_pkl_path = data_pkl_dir_path.joinpath(REVIEWS_PKL_FILENAME)

In [141]:
safeload_reviews_df = pd.read_csv(reviews_in_path,
                                        dtype='string',
                                        na_values=[],
                                        keep_default_na=False
                                        )
safeload_reviews_df.columns

Index(['review.uid', 'user.uid', 'text'], dtype='object')

In [142]:
safeload_reviews_df.shape

(105710, 3)

In [143]:
print(safeload_reviews_df["review.uid"].unique().shape)
# I expect this number to be the same as the dimension of the shape, and that's what happens

print(safeload_reviews_df["review.uid"].isnull().sum())
#I expect this number to be zero, and that's what happens

print(safeload_reviews_df["review.uid"].duplicated().sum())
#I expect this number to be zero, and that's what happenS

(105710,)
0
0


In [144]:
print(safeload_reviews_df['user.uid'].isnull().sum())
print(safeload_reviews_df["user.uid"].unique().shape)
# we have no user that has written more than one review

0
(105710,)


In [145]:
print(safeload_reviews_df["text"].isnull().sum())
print(safeload_reviews_df["text"].unique().shape)
#we have someone that has written the same review text (the dimesion is lower than the shape)

0
(93429,)


In [146]:
# find duplicates in "text"
duplicates = safeload_reviews_df[safeload_reviews_df.duplicated(subset="text", keep=False)]

# Print 5 couples
groups = duplicates.groupby("text")
for i, (text, group) in enumerate(groups):
    if i >= 5:  
        break
    print(group["text"].values[:2])  # stampo le prime 2 occorrenze


<StringArray>
['"Bounty" bars consist of a soft, almost molten center of sweet coconut coated in an outer layer of milk chocolate. You can easily buy them "over the counter" here in Australia. But if that\'s not the case where you are, and you don\'t mind a diet rich in junk food, then yes, this is a candy bar worth going to the trouble of ordering over the internet.<br /><br />Note that here on Amazon you can purchase bounty bars in boxes with varying numbers of bars inside, so it is worth your while to crunch the numbers and think about what package offers the best value for you. At the same time, it may not be entirely in the best interests of your health to have _too_ many of these things lying around at home all at once!<br /><br />Theo.', '"Bounty" bars consist of a soft, almost molten center of sweet coconut coated in an outer layer of milk chocolate. You can easily buy them "over the counter" here in Australia. But if that\'s not the case where you are, and you don\'t mind a di

In [147]:
reviews_df = pd.read_csv(reviews_in_path,
                       keep_default_na=False,
                       na_values=NA_VALUES,
                       dtype={
                           'review.uid': 'string',
                           'user.uid': 'string',
                           'text': 'string'
                       }
                       )

reviews_df = reviews_df.drop_duplicates(subset="text", keep="first")
reviews_df = reviews_df.reset_index(drop=True)

# ------------------------------
# CHECK PK VALIDITY: review.uid
# ------------------------------

#null check
display(
    reviews_df
    .loc[lambda tbl: tbl['review.uid'].isnull()]
    .assign(aux=1.0)
    .shape[0]
)


# uniqueness check
display(
    reviews_df
    .assign(aux=1.0)
    .groupby(['review.uid'], dropna=False)
    .agg(num_rows=('aux', pd.Series.count))
    .loc[lambda tbl: tbl['num_rows'] > 1]
)
display(
    reviews_df
    .loc[lambda tbl: tbl['user.uid'].isnull()]
    .assign(aux=1.0)
    .shape[0]
)

0

,num_rows
review.uid,


0

In [148]:
# CHECK Foreign Key VALIDITY

(reviews_df
 [['user.uid']]
 .loc[lambda tbl: tbl['user.uid'].notnull()]
 .drop_duplicates()
 .merge(users_df[['user.uid']].assign(in_users=1.0),
        how='left',
        on='user.uid'
        )
 .assign(aux=1.0)
 .groupby(['in_users'], dropna=False)
 .agg(num_rows=('aux', pd.Series.count))
 .loc[lambda tbl: tbl['num_rows'] > 1]
)

,num_rows
in_users,
1.0,93429


## Load Labelled Reviews

In [149]:
REVIEWS_LABELLED_IN_FILENAME = 'reviews_labelled.csv'
REVIEWS_LABELLED_PKL_FILENAME = 'reviews_labelled.pkl'

reviews_labelled_in_path = data_in_dir_path.joinpath(REVIEWS_LABELLED_IN_FILENAME) 
reviews_labelled_pkl_path = data_pkl_dir_path.joinpath(REVIEWS_LABELLED_PKL_FILENAME)
safeload_reviews_labelled_df = pd.read_csv(reviews_labelled_in_path,
                                        dtype='string',
                                        na_values=[],
                                        keep_default_na=False
                                        )
safeload_reviews_labelled_df.columns

Index(['labelled_review.uid', 'text', 'sentiment_label'], dtype='object')

In [150]:
safeload_reviews_labelled_df.shape

(462744, 3)

In [151]:
print(safeload_reviews_labelled_df["labelled_review.uid"].unique().shape)
# I expect this number to be the same as the dimension of the shape, and that's what happens

print(safeload_reviews_labelled_df["labelled_review.uid"].isnull().sum())
#I expect this number to be zero, and that's what happens

print(safeload_reviews_labelled_df["labelled_review.uid"].duplicated().sum())
#I expect this number to be zero, and that's what happenS

(462744,)
0
0


In [152]:
print(safeload_reviews_labelled_df['sentiment_label'].isnull().sum())
print(safeload_reviews_labelled_df['sentiment_label'].unique())

0
<StringArray>
['neutral', 'positive', 'negative']
Length: 3, dtype: string


In [153]:
print(safeload_reviews_labelled_df["text"].isnull().sum())
print(safeload_reviews_labelled_df["text"].unique().shape)

0
(327522,)


In [154]:
#we have someone that has written the same review text (the dimesion is lower than the shape)
# find duplicates in "text"
duplicates = safeload_reviews_labelled_df[safeload_reviews_labelled_df.duplicated(subset="text", keep=False)]

# Print 5 couples
groups = duplicates.groupby("text")
for i, (text, group) in enumerate(groups):
    if i >= 5:  
        break
    print(group["text"].values[:2])  # stampo le prime 2 occorrenze

<StringArray>
['"4C Totally Light" is one of the very few "sugar free" products of it\'s type to use safer and better tasteing sucralose(Splenda) over Aspertame.<br /><br />Very tasty drink, especially when served over ice!', '"4C Totally Light" is one of the very few "sugar free" products of it\'s type to use safer and better tasteing sucralose(Splenda) over Aspertame.<br /><br />Very tasty drink, especially when served over ice!']
Length: 2, dtype: string
<StringArray>
['"BUFFY" LOOKS FORWARD TO HER "TOY" EVERY AFTERNOON.  MY BIGGEST FEAR IS RUNNING OUT OF REFILLS BEFORE I CAN ORDER AND RECEIVE MORE.', '"BUFFY" LOOKS FORWARD TO HER "TOY" EVERY AFTERNOON.  MY BIGGEST FEAR IS RUNNING OUT OF REFILLS BEFORE I CAN ORDER AND RECEIVE MORE.']
Length: 2, dtype: string
<StringArray>
['"Blends smooth and creamy for a sweet tasting protein"  is in the item description. I don\'t know who wrote this but they must not have ever smelled or tasted this stuff! Based on the description, I purchased a f

In [155]:
reviews_labelled_df = pd.read_csv(reviews_labelled_in_path,
                       keep_default_na=False,
                       na_values=NA_VALUES,
                       dtype={
                           'labelled_review.uid': 'string',
                           'text': 'string',
                           'sentiment_label': 'string'
                       }
                       )

reviews_labelled_df = reviews_labelled_df.drop_duplicates(subset="text", keep="first")
reviews_labelled_df = reviews_labelled_df.reset_index(drop=True)


# ------------------------------
# CHECK PK VALIDITY: review.uid
# ------------------------------

#null check
display(
    reviews_labelled_df
    .loc[lambda tbl: tbl['labelled_review.uid'].isnull()]
    .assign(aux=1.0)
    .shape[0]
)

0

# Save

In [156]:
# Save Cleaned Dataset

with users_pkl_path.open('wb') as fh:
    dill.dump(users_df, fh)
print(f"Save users data in {users_pkl_path.as_posix()}")

with profiles_pkl_path.open('wb') as fh:
    dill.dump(profiles_df, fh)
print(f"Save profiles data in {profiles_pkl_path.as_posix()}")

with cards_pkl_path.open('wb') as fh:
    dill.dump(cards_df, fh)
print(f"Save cards data in {cards_pkl_path.as_posix()}")

with orders_pkl_path.open('wb') as fh:
    dill.dump(orders_df, fh)
print(f"Save orders data in {orders_pkl_path.as_posix()}")

with order_details_pkl_path.open('wb') as fh:
    dill.dump(order_details_df, fh)
print(f"Save order details data in {order_details_pkl_path.as_posix()}")

with reviews_pkl_path.open('wb') as fh:
    dill.dump(reviews_df, fh)
print(f"Save reviews data in {reviews_pkl_path.as_posix()}")

with reviews_labelled_pkl_path.open('wb') as fh:
    dill.dump(reviews_labelled_df, fh)
print(f"Save reviews labelled data in {reviews_labelled_pkl_path.as_posix()}")

Save users data in C:/Users/davyt/Desktop/Bicocca/Marketing Analytics/unimib_snowit_project/data_loaded/users.pkl
Save profiles data in C:/Users/davyt/Desktop/Bicocca/Marketing Analytics/unimib_snowit_project/data_loaded/profiles.pkl
Save cards data in C:/Users/davyt/Desktop/Bicocca/Marketing Analytics/unimib_snowit_project/data_loaded/cards.pkl
Save orders data in C:/Users/davyt/Desktop/Bicocca/Marketing Analytics/unimib_snowit_project/data_loaded/orders.pkl
Save order details data in C:/Users/davyt/Desktop/Bicocca/Marketing Analytics/unimib_snowit_project/data_loaded/order_details.pkl
Save reviews data in C:/Users/davyt/Desktop/Bicocca/Marketing Analytics/unimib_snowit_project/data_loaded/reviews.pkl
Save reviews labelled data in C:/Users/davyt/Desktop/Bicocca/Marketing Analytics/unimib_snowit_project/data_loaded/reviews_labelled.pkl
